**IMPORTAR LIBRERIAS**

---

In [1]:
# Librerias de manejo de datos
import pandas as pd

In [2]:
# Importar conexión
from db_connection import get_connection

# Conectarse a la base de datos
conexion = get_connection()
cursor = conexion.cursor()


✅ Conexión exitosa a la base de datos


In [3]:
# Importar librerías para la conexión a la base de datos con SQLAlchemy
from db_connection import get_engine

engine = get_engine()

In [4]:
# 📌 Obtener todas las tablas
cursor.execute("SHOW TABLES;")
tablas = [fila[0] for fila in cursor.fetchall()]
print(f"Total de tablas encontradas: {len(tablas)}")


Total de tablas encontradas: 50


---

**FUNCIONES**

---

------

**FILTROS PARA SEGREGAR Y ELIMINAR LAS TABLAS QUE NO SON ÚTILES**


---

In [5]:
#Tablas posiblemente innecesarias por el nombre y breve análisis*
# Imprimir los nombres de las tablas restantes
for tabla in tablas:
    print(tabla)


accounting_account_balances
accounting_accounts
accounting_movements
accounting_voucher_items
accounting_voucher_types
accounting_vouchers
billing_numberings
contact_relationships
contact_statements
contacts
contract_salary_history
costs_and_expenses
costs_and_expenses_categories
coupons
document_items
documents
ecommerce_legal_info
ecommerce_purchase_orders
employee_contracts
employee_positions
employees
fixed_asset_depreciations
fixed_assets
fixed_assets_groups
headquarter_warehouses
headquarters
inventory_adjustments
inventory_groups
item_balance
item_categories
item_depreciations
item_kardex
item_subcategories
item_variations
items
payment_conditions
payments
payroll_deductions
payroll_details
payroll_incomes
payroll_providers
payrolls
price_lists
retention_concepts
retentions
retentions_applied
retentions_certificates
taxes
warehouse_transfer_logs
warehouses


*ANÁLISIS Y CONCLUSIONES DE LA INFORMACIÓN OBTENIDA*

**Tablas clave para análisis de gestión empresarial (💡 Mantener)**

Estas son probablemente las más útiles para análisis de KPIs, dashboards o visualización de negocio:

*Categoría	//  Tablas clave*

📦 Inventario	= items, item_balance, item_kardex, warehouses, inventory_adjustments, item_categories, item_subcategories, item_variations

💼 Ventas / Documentos	= documents, document_items, contacts (clientes/proveedores)

🧾 Contabilidad	= accounting_vouchers, accounting_voucher_items, accounting_accounts, accounting_movements, retentions, retention_concepts, retentions_applied, accounting_account_balances

👥 Empleados / Nómina	= employees, employee_contracts, payrolls, payroll_details, payroll_incomes, payroll_deductions

🛒 Compras	= ecommerce_purchase_orders

📊 Precios	= price_lists

🏪 Organización	= headquarters, user_headquarters, roles, user_roles, company_areas

🧾 Impuestos	= taxes

🏭 Activos fijos	= fixed_assets, fixed_asset_depreciations, fixed_assets_groups


**Relevancia baja o dependiente del contexto (🤔 Revisar)**

*Estas podrían ser útiles dependiendo del foco del proyecto (clientes, pedidos, legal, e-commerce):*

contact_relationships, contact_accounts, contact_statements

coupons, coupon_groups (si hay ecommerce)

employee_positions, contract_salary_history

plan_electronic_documents, plan_system_controller

ecommerce_legal_info

costs_and_expenses, costs_and_expenses_categories

radian_documents (puede ser de facturación electrónica)

dining_tables (¿restaurante? puede ser irrelevante)

system_counters, system_restrictions

In [6]:
# Lista de tablas candidatas a eliminar
tablas_candidatas = [
    "api_access_tokens",
    "electronic_payroll_test_set",
    "migrations",
    "oauth_clients",
    "paynilo",
    "role_permissions",
    "template_versions",
    "templates",
    "user_data",
    "values_x_item",
    "warehouse_transfer_logs",
    "billing_numberings",
    "custom_fields",
    "configurations",
    "system_counters",
    "system_restrictions"
]

---

*tablas a eliminar según sus columnas y número de filas*

In [7]:
# Script para limpiar y aplanar columnas JSON en 'employees' (VERSIÓN DIRECTA CON FK_CHECKS DESHABILITADO)

import pandas as pd
import json
import numpy as np
from db_connection import get_engine, get_connection

print("\n--- Bloque 6 (VERSIÓN DIRECTA): Limpieza de JSONs en 'employees' con Pandas ---")

conexion = None
cursor = None
try:
    print("Reestableciendo conexión a la base de datos para el bloque de 'employees'...")
    conexion = get_connection()
    cursor = conexion.cursor()
    engine = get_engine()
    print("✅ Conexión y cursor reestablecidos exitosamente para 'employees'.")
    print("✅ Engine de base de datos obtenido.")

    # --- PASO CRÍTICO: Deshabilitar temporalmente las verificaciones de claves foráneas ---
    # Esto permite realizar el ALTER TABLE sin problemas de IDs huérfanos.
    print("\nDeshabilitando temporalmente las verificaciones de claves foráneas...")
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
    print("✅ Verificaciones de claves foráneas deshabilitadas.")

    # --- PASO 1: Modificar la estructura de la tabla 'employees' ---
    # Convertir las columnas JSON a VARCHAR(255) NULL
    print("\nModificando las columnas JSON a VARCHAR(255) NULL...")
    alter_table_sql = """
        ALTER TABLE employees
        MODIFY COLUMN identity_document_type VARCHAR(255) NULL,
        MODIFY COLUMN country VARCHAR(255) NULL,
        MODIFY COLUMN region VARCHAR(255) NULL,
        MODIFY COLUMN city VARCHAR(255) NULL,
        MODIFY COLUMN emergency_contact VARCHAR(255) NULL,
        MODIFY COLUMN payment_method VARCHAR(255) NULL,
        MODIFY COLUMN bank VARCHAR(255) NULL,
        MODIFY COLUMN bank_account_type VARCHAR(255) NULL;
    """
    try:
        cursor.execute(alter_table_sql)
        conexion.commit() # Confirmar el cambio de estructura
        print("✅ Columnas JSON modificadas a VARCHAR(255) NULL exitosamente.")
    except Exception as e:
        conexion.rollback()
        print(f"❌ Error al modificar las columnas JSON: {e}")
        raise # Re-lanza el error para detener el script

    # --- PASO CRÍTICO: Re-habilitar las verificaciones de claves foráneas ---
    # Es FUNDAMENTAL re-habilitarlas inmediatamente después.
    print("\nRe-habilitando las verificaciones de claves foráneas...")
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
    print("✅ Verificaciones de claves foráneas re-habilitadas.")

    # 1. Cargar la tabla 'employees' completa en un DataFrame de Pandas
    print("\nCargando la tabla 'employees' en un DataFrame de Pandas...")
    df_employees = pd.read_sql("SELECT * FROM employees;", engine)
    print(f"DataFrame de 'employees' cargado. Registros: {len(df_employees)}")

    # Lista de columnas que eran JSON (ahora son VARCHAR en la DB)
    json_columns = [
        'identity_document_type', 'country', 'region', 'city',
        'emergency_contact', 'payment_method', 'bank', 'bank_account_type'
    ]

    # 2. Iterar sobre cada columna (ahora VARCHAR, pero con contenido JSON previo) y aplanarla
    print("\nIniciando aplanamiento de columnas (previamente JSON) en Pandas...")

    def extract_json_value(json_str):
        # Manejar casos donde la columna ya fue aplanada o es None/NaN
        if pd.isna(json_str) or not isinstance(json_str, str) or not json_str.strip():
            return None
        # Si la cadena no parece JSON (ej. ya es un valor aplanado), devolverla tal cual
        if not json_str.strip().startswith('{') and not json_str.strip().startswith('['):
            return json_str # Ya está aplanado, o es un string simple no JSON
        try:
            data = json.loads(json_str)
            if isinstance(data, dict):
                if 'description' in data and data['description'] is not None:
                    return str(data['description'])
                elif 'name' in data and data['name'] is not None:
                    return str(data['name'])
                elif 'code' in data and data['code'] is not None:
                    return str(data['code'])
            return None
        except json.JSONDecodeError:
            # Si no es un JSON válido, pero es una cadena, devolverla.
            # Esto maneja el caso donde la columna ya fue aplanada manualmente o contiene basura.
            return json_str if json_str.strip() else None
        except Exception as e:
            return None

    for col in json_columns:
        print(f"Procesando columna: '{col}'...")
        if col in df_employees.columns:
            df_employees[col] = df_employees[col].astype(str)
            df_employees[col] = df_employees[col].apply(extract_json_value)
            print(f"   Columna '{col}' aplanada a valor extraído o NULL.")
        else:
            print(f"   Advertencia: Columna '{col}' no encontrada en el DataFrame.")
    print("✅ Aplanamiento de columnas completado.")

    # 3. Manejo de valores nulos y tipos de datos para la actualización
    print("\nPreparando DataFrame para la actualización (manejo de nulos y tipos)...")

    # Volvemos a obtener la descripción de la tabla para asegurarnos de que los tipos son los nuevos (VARCHAR)
    cursor.execute("DESCRIBE `employees`;")
    db_columns_info = cursor.fetchall()
    db_column_names = [col_info[0] for col_info in db_columns_info]
    db_column_types = {col_info[0]: col_info[1] for col_info in db_columns_info}

    valid_columns_for_update = [
        col for col in df_employees.columns
        if isinstance(col, str) and col != 'id' and col in db_column_names
    ]

    for col in df_employees.columns:
        if col in valid_columns_for_update or col == 'id':
            if df_employees[col].dtype == 'object':
                df_employees[col] = df_employees[col].replace({np.nan: None, 'nan': None, '': None})
            elif pd.api.types.is_numeric_dtype(df_employees[col]) and df_employees[col].isnull().any():
                if pd.api.types.is_integer_dtype(df_employees[col]):
                     df_employees[col] = df_employees[col].astype(pd.Int64Dtype())
                df_employees[col] = df_employees[col].replace({np.nan: None})

            if pd.api.types.is_bool_dtype(df_employees[col]):
                df_employees[col] = df_employees[col].map({True: 1, False: 0, None: None, np.nan: None})

            if pd.api.types.is_string_dtype(df_employees[col]) and col in db_column_types:
                mysql_type = db_column_types[col].lower()
                if 'varchar' in mysql_type:
                    try:
                        max_len = int(mysql_type.split('(')[1].split(')')[0])
                        long_strings = df_employees[df_employees[col].astype(str).str.len() > max_len][col]
                        if not long_strings.empty:
                            print(f"   ⚠️ ADVERTENCIA: Columna '{col}' tiene strings más largos que {max_len} caracteres.")
                            print(f"     Primeros 5 valores problemáticos: {long_strings.head().tolist()}")
                    except (IndexError, ValueError):
                        pass

    print("✅ DataFrame preparado para la actualización.")

    # 4. Actualizar los registros con los datos limpios del DataFrame
    print("\n--- Actualizando registros en 'employees' con datos limpios ---")

    set_clauses = [f"`{col}` = %s" for col in valid_columns_for_update]
    update_sql = f"UPDATE `employees` SET {', '.join(set_clauses)} WHERE `id` = %s;"

    print(f"DEBUG: SQL de UPDATE generado: {update_sql[:150]}...")
    print(f"DEBUG: Columnas que se actualizarán: {valid_columns_for_update}")

    updated_count = 0
    errors_encountered = 0
    for index, row in df_employees.iterrows():
        update_values = []
        for col in valid_columns_for_update:
            val = row[col]
            if pd.isna(val) or (isinstance(val, (float, np.float64)) and np.isnan(val)):
                update_values.append(None)
            elif isinstance(val, (np.int64, np.int32)):
                update_values.append(int(val))
            elif isinstance(val, (np.bool_)):
                update_values.append(bool(val))
            else:
                update_values.append(val)

        update_values.append(row['id'])

        try:
            cursor.execute(update_sql, tuple(update_values))
            updated_count += 1
            if updated_count % 5 == 0:
                print(f"   Progreso: {updated_count}/{len(df_employees)} registros procesados...")
        except Exception as e:
            errors_encountered += 1
            print(f"\n❌❌ ERROR CRÍTICO al actualizar el registro con ID {row['id']}: {e}")
            print(f"   SQL: {update_sql}")
            print(f"   Valores problemáticos (ID={row['id']}): {tuple(update_values)}")
            raise

    if errors_encountered > 0:
        conexion.rollback()
        print(f"❌ Actualización finalizada con {errors_encountered} errores. Transacción revertida.")
    else:
        conexion.commit()
        print(f"✅ Actualización de {updated_count} registros en la tabla 'employees' completada exitosamente.")

    print("\n✅ Limpieza y aplanamiento de JSONs en 'employees' finalizada exitosamente.")

except Exception as e:
    print(f"\n❌❌ Ocurrió un error general NO CAPTURADO en el bucle principal: {e}")
    print("Por favor, revisa la conexión a la base de datos o los datos en sí.")
    try:
        if conexion and conexion.is_connected():
            conexion.rollback()
            print("Transacción revertida debido a un error.")
    except Exception as rollback_e:
        print(f"❌ Error adicional al intentar revertir la transacción: {rollback_e}")
finally:
    # Asegúrate de re-habilitar las FKs incluso si hay un error en el try-except.
    # Esto es CRÍTICO para mantener la integridad de tu base de datos.
    if cursor is not None:
        try:
            cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
            print("✅ Verificaciones de claves foráneas re-habilitadas en el bloque 'finally'.")
        except Exception as e:
            print(f"❌ Advertencia: No se pudieron re-habilitar las FKs en finally: {e}")

    # La conexión a la base de datos se mantiene abierta según la solicitud.
    print("La conexión a la base de datos se mantiene abierta según la solicitud.")


--- Bloque 6 (VERSIÓN DIRECTA): Limpieza de JSONs en 'employees' con Pandas ---
Reestableciendo conexión a la base de datos para el bloque de 'employees'...
✅ Conexión exitosa a la base de datos
✅ Conexión y cursor reestablecidos exitosamente para 'employees'.
✅ Engine de base de datos obtenido.

Deshabilitando temporalmente las verificaciones de claves foráneas...
✅ Verificaciones de claves foráneas deshabilitadas.

Modificando las columnas JSON a VARCHAR(255) NULL...
✅ Columnas JSON modificadas a VARCHAR(255) NULL exitosamente.

Re-habilitando las verificaciones de claves foráneas...
✅ Verificaciones de claves foráneas re-habilitadas.

Cargando la tabla 'employees' en un DataFrame de Pandas...
DataFrame de 'employees' cargado. Registros: 38

Iniciando aplanamiento de columnas (previamente JSON) en Pandas...
Procesando columna: 'identity_document_type'...
   Columna 'identity_document_type' aplanada a valor extraído o NULL.
Procesando columna: 'country'...
   Columna 'country' aplana

**Tablas candidatas fuertes para eliminación (por contenido y nombre)**

Estas tablas tienen pocos registros, estructuras irrelevantes para KPIs o funciones accesorias que no impactan directamente en la gestión:

*Tabla	// Motivo principal*

api_access_tokens	Manejo técnico de tokens

electronic_payroll_test_set	Datos de prueba (test set)

migrations	Solo para migraciones del sistema

oauth_clients	Seguridad/autenticación interna

paynilo	Integración de pagos externa

role_permissions	Control de permisos internos

template_versions	Versiones de plantillas, sin datos

templates	Estructura visual, no analítica

user_data	Solo datos de área/cargo del usuario

custom_fields	Campos personalizados (metadatos)

system_counters	Estadísticas internas del sistema

system_restrictions	Restricciones de uso/licencia

values_x_item	Datos muy técnicos por item, posiblemente redundantes

warehouse_transfer_logs	Registro técnico, no necesariamente útil para KPIs

----

*CUARTO FILTRO: Investigar mas a fondo las tablas dudosas*

**⚠️ Tablas dudosas (verificar con .info() antes de decidir)**

Estas tienen algo más de contenido o potencial analítico dependiendo del negocio:

*billing_numberings:* Tiene 155 registros. Podría contener información útil para facturación si los números se asocian a documentos.

*configurations:* Solo 25 registros, pero hay que ver si son configuraciones claves que afectan reportes.



In [8]:
# Obtener el esquema de cada tabla para decidir si son candidatas a eliminación
tablas_dudosas = ['billing_numberings', 'configurations'] # Agregar mas tablas si es necesario

for tabla in tablas_dudosas:
    print(f"\n📋 Tabla: {tabla}")
    try:
        df = pd.read_sql(f"SELECT * FROM {tabla}", con=engine)
        print("▶️ .info():")
        print(df.info())
        print("▶️ Primeras filas:")
        display(df.head())
    except Exception as e:
        print(f"❌ Error al cargar {tabla}: {e}")



📋 Tabla: billing_numberings
▶️ .info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      50 non-null     int64         
 1   document_type_id        50 non-null     int64         
 2   pos_numbering           50 non-null     int64         
 3   name                    50 non-null     object        
 4   prefix                  50 non-null     object        
 5   initial_number          50 non-null     int64         
 6   current_number          50 non-null     int64         
 7   final_number            22 non-null     float64       
 8   valid_since             22 non-null     object        
 9   valid_until             22 non-null     object        
 10  validity                22 non-null     float64       
 11  preferred               50 non-null     int64         
 12  electronic_

,id,document_type_id,pos_numbering,name,prefix,initial_number,current_number,final_number,valid_since,valid_until,...,tech_key,used,template,headquarter_id,warehouse_id,status,user_id,created_at,updated_at,contingency
0,1,11,0,Remisiones,,0,56,NaN,None,None,...,None,1,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,10,NaT,2024-05-24 09:54:00,0
1,2,12,0,Cotizaciones,,0,91,NaN,None,None,...,None,1,"{""id"": 3, ""name"": ""POS"", ""template_type"": ""1""}",1,1,1,10,NaT,2024-07-13 11:53:24,0
2,3,13,0,Recibos de Caja,,0,506,NaN,None,None,...,None,1,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,10,NaT,2024-07-17 10:58:41,0
3,4,14,0,Comprobantes de Pago,,0,113,NaN,None,None,...,None,1,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,0,NaT,2024-07-17 11:00:31,0
4,5,15,0,Órdenes de Compra,,0,23,NaN,None,None,...,None,1,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,0,NaT,2024-04-03 16:10:15,0



📋 Tabla: configurations
❌ Error al cargar configurations: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'novateam.configurations' doesn't exist
[SQL: SELECT * FROM configurations]
(Background on this error at: https://sqlalche.me/e/20/f405)


---

**TABLAS FINALES A ELIMINAR DESPUÉS DE TODOS LOS FILTROS Y ANÁLISIS INDIVIDUAL**

Falta escoger y seleccionar individualmente 

In [9]:
tablas_a_eliminar = [
    "api_access_tokens",             # Manejo técnico de tokens
    "electronic_payroll_test_set",   # Datos de prueba (test set)
    "migrations",                    # Solo para migraciones del sistema
    "oauth_clients",                 # Seguridad/autenticación interna
    "paynilo",                       # Integración de pagos externa
    "role_permissions",             # Control de permisos internos
    "template_versions",            # Versiones de plantillas, sin datos
    "templates",                    # Estructura visual, no analítica
    "user_data",                    # Solo datos de área/cargo del usuario
    "custom_fields",                # Campos personalizados (metadatos)
    "system_counters",              # Estadísticas internas del sistema
    "system_restrictions",          # Restricciones de uso/licencia
    "values_x_item",                # Datos técnicos por ítem
    "user_roles",
    "user_data",
    "user_headquarters",
    "term_and_conditions,",
    "severance_payments",
    "roles", 
    "role_permissions",
    "radian_events",
    "radian_documents",
    "plan_system_controller",
    "plan_electronic_documents",
    "api_access_tokens",
    "configurations",
    "company_areas",
    "consolidated_retention_certificates",
    "contact_accounts",
    "contact_relationship",
    "contact_statement",
    "contact",
    "custom_fields",
    "dining_tables",
    "ecomerce_legal_info",
    "ecomerce_purchase_orders",
    "electronic_payroll_test_set", 
    "electronic_contracts",
    "migraciones ]"     # Registro técnico, no útil para KPIs

]




In [10]:
# ⚠️ Solo si tienes respaldo y estás en entorno seguro
# ELIMINAR TABLAS FINALES ¡OJO HAY QUE ESCOGER ANTES DE EJECUTAR!

# Desactivar restricciones !NO SE HACE ANTES DE VERIFICAR QUE NO GENERA NINGÚN DAÑO!
# cursor.execute("SET FOREIGN_KEY_CHECKS=0;")

# borrar_lista_tablas(tablas_a_eliminar)

# Volver a activar restricciones !NO SE HACE ANTES DE VERIFICAR QUE NO GENERA NINGÚN DAÑO!
#cursor.execute("SET FOREIGN_KEY_CHECKS=1;")
#

---


**TABLAS ASIGNADAS AL ANÁLISIS DE RH - Alexandra García **

A continuación, se irá explorando una a una, las tablas y su contenido para visualizar qué tan viables son para el ejercicio propuesto, se realizará limpieza de datos, se eliminarán nulls según convenga o se reemplazarán por n/a.  

employee_positions, employees, , employee_contracts, contract_salary_history, payrolls, payroll_details, payroll_incomes, payroll_deductions, payroll_providers, costs_and_expenses, costs_and_expenses_categories, headquarters, coupons. 

por cuestiones de tiempo, sólo se logró realizar ETL completo a las tres primeras.
Eso lo verán a continuación: 



##TABLA 1: EMPLOYEE_POSITIONS  O Posiciones de empleados ##


In [11]:
# Conectarse a la base de datos
conexion = get_connection()
cursor = conexion.cursor()

try:
    # Obtener la descripción de las columnas
    cursor.execute("DESCRIBE employee_positions;")
    columnas_employee_positions = cursor.fetchall()

    print("Columnas de 'employee_positions':")
    for columna in columnas_employee_positions:
        print(f"  - {columna[0]} ({columna[1]})")

except Exception as e:
    print(f"Error al obtener columnas de 'employee_positions': {e}")
finally:
    # Cerrar el cursor y la conexión
    if cursor:
        cursor.close()
    if conexion:
        conexion.close()

✅ Conexión exitosa a la base de datos
Columnas de 'employee_positions':
  - id (int unsigned)
  - name (varchar(255))
  - description (varchar(500))
  - salary (double(16,2))
  - status (tinyint(1))
  - user_id (int unsigned)
  - created_at (timestamp)
  - updated_at (timestamp)


Aquí hay una versión muy concisa y sustanciosa de la celda, enfocada en la información clave para el análisis, sin la verbosidad anterior.

1. 💼 Tabla: employee_positions
Exploración de Datos Clave 🔍

Esta tabla describe los puestos de trabajo en la organización. Esencial para la estructura organizacional y la gestión de la compensación.

Columnas principales:

id: Identificador único de cada posición.
name: Nombre del puesto (ej., "Gerente de Proyectos").
description: Descripción del rol.
salary: Salario de referencia asociado a la posición.
status: Estado de la posición (activa/inactiva).
Consideraciones para el análisis:

Estandarización de name: Unificar nombres de puestos similares.
Manejo de nulos en salary: Definir cómo tratar los valores faltantes.

In [12]:
# Script para la verificación inicial de datos en 'employee_positions'
try:
    # 1. Obtener el número total de registros
    cursor.execute("SELECT COUNT(*) FROM employee_positions;")
    total_registros = cursor.fetchone()[0]
    print(f"\nTotal de registros en 'employee_positions': {total_registros}")

    # 2. Verificar valores nulos por columna
    print("\nValores nulos por columna en 'employee_positions':")
    for columna_info in columnas_employee_positions: # Usamos 'columnas_employee_positions' que ya obtuvimos
        nombre_columna = columna_info[0]
        cursor.execute(f"SELECT COUNT(*) FROM employee_positions WHERE {nombre_columna} IS NULL;")
        nulos_count = cursor.fetchone()[0]
        if nulos_count > 0:
            print(f"  - {nombre_columna}: {nulos_count} nulos")
        else:
            print(f"  - {nombre_columna}: 0 nulos")

    # 3. Verificar filas duplicadas (basado en todas las columnas, o solo el ID si es la clave primaria)
    # Asumimos que 'id' debería ser único
    cursor.execute("SELECT COUNT(id) - COUNT(DISTINCT id) FROM employee_positions;")
    duplicados_id = cursor.fetchone()[0]
    print(f"\nFilas duplicadas basadas en 'id' en 'employee_positions': {duplicados_id}")

    
    # 4. Obtener valores únicos para columnas específicas
    print("\nValores únicos en columnas clave de 'employee_positions':")

    # Para 'name'
    cursor.execute("SELECT DISTINCT name FROM employee_positions LIMIT 10;") # Limitar para no cargar demasiados si hay muchos
    nombres_unicos = [fila[0] for fila in cursor.fetchall()]
    print(f"  - name (primeros 10 únicos): {nombres_unicos}")
    cursor.execute("SELECT COUNT(DISTINCT name) FROM employee_positions;")
    count_nombres_unicos = cursor.fetchone()[0]
    print(f"  - Total de nombres únicos: {count_nombres_unicos}")

    # Para 'status'
    cursor.execute("SELECT DISTINCT status FROM employee_positions;")
    estados_unicos = [fila[0] for fila in cursor.fetchall()]
    print(f"  - status: {estados_unicos}")


except Exception as e:
    print(f"Ocurrió un error durante la verificación inicial de datos en 'employee_positions': {e}")

Ocurrió un error durante la verificación inicial de datos en 'employee_positions': 2055: Cursor is not connected


**Bloque 2: Limpieza y Transformación de la 1ra tabla: employee_positions**

In [13]:
conexion = get_connection()
cursor = conexion.cursor()

try:
    print("\nIniciando limpieza y transformación para 'employee_positions'...")

    # 1. Corrección de errores tipográficos en la columna 'name'
    # Detectamos 'Secretria' que debería ser 'Secretaria'
    print("Corrigiendo error tipográfico en la columna 'name': 'Secretria' a 'Secretaria'...")
    cursor.execute("""
        UPDATE employee_positions
        SET name = 'Secretaria'
        WHERE name = 'Secretria';
    """)
    conexion.commit() # Confirmar los cambios en la base de datos
    print("Corrección de 'name' completada.")

    # 2. Manejo de valores nulos en la columna 'description'
    # Rellenar con 'N/A' según lo acordado.
    print("Rellenando valores nulos en la columna 'description' con 'N/A'...")
    cursor.execute("""
        UPDATE employee_positions
        SET description = 'N/A'
        WHERE description IS NULL;
    """)
    conexion.commit() # Confirmar los cambios
    print("Relleno de 'description' completado.")

    print("\nLimpieza y transformación de 'employee_positions' finalizada.")

except Exception as e:
    print(f"Ocurrió un error durante la limpieza y transformación de 'employee_positions': {e}")
    conexion.rollback() # Revertir cambios en caso de error

✅ Conexión exitosa a la base de datos



Iniciando limpieza y transformación para 'employee_positions'...
Corrigiendo error tipográfico en la columna 'name': 'Secretria' a 'Secretaria'...
Corrección de 'name' completada.
Rellenando valores nulos en la columna 'description' con 'N/A'...
Relleno de 'description' completado.

Limpieza y transformación de 'employee_positions' finalizada.


**TABLA 2: EMPLOYEES o EMPLEADOS**


In [14]:
conexion = get_connection()
cursor = conexion.cursor()

# Script para obtener la información de las columnas de 'employees'
try:
    # Obtener la descripción de las columnas de la tabla 'employees'
    cursor.execute("DESCRIBE employees;")
    columnas_employees = cursor.fetchall()

    print("Columnas de la tabla 'employees':")
    for columna in columnas_employees:
        # Cada 'columna' es una tupla con (nombre_columna, tipo_dato, ...)
        print(f"  - {columna[0]} ({columna[1]})")

except Exception as e:
    print(f"Ocurrió un error al intentar obtener las columnas de 'employees': {e}")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'employees':
  - id (int unsigned)
  - uuid (char(36))
  - identity_document_type (varchar(255))
  - document_number (varchar(25))
  - birth_date (date)
  - first_name (varchar(50))
  - other_name (varchar(50))
  - first_surname (varchar(50))
  - second_surname (varchar(50))
  - full_name (varchar(255))
  - phone (varchar(25))
  - email (varchar(75))
  - country (varchar(255))
  - region (varchar(255))
  - city (varchar(255))
  - address (varchar(255))
  - headquarter_id (int unsigned)
  - company_area_id (int unsigned)
  - employee_position_id (int unsigned)
  - emergency_contact (varchar(255))
  - payment_type (tinyint unsigned)
  - payment_method (varchar(255))
  - bank (varchar(255))
  - bank_account_type (varchar(255))
  - bank_account_number (varchar(255))
  - status (tinyint(1))
  - settled (tinyint(1))
  - user_id (int unsigned)
  - created_at (timestamp)
  - updated_at (timestamp)
  - latitude (decimal(10,7))
  - longi

Bloque 4: Verificación inicial de nulos, duplicados y valores únicos en employees


In [15]:
# Importar conexión
from db_connection import get_connection

# Conectarse a la base de datos
conexion = get_connection()
cursor = conexion.cursor()

# Script para la limpieza y transformación inicial de 'employees' (VERSION FINAL)
print("\n--- Bloque 5: Limpieza y Transformación Inicial de 'employees' ---")

try:
    print("Iniciando operaciones de limpieza en 'employees'...")

    # 1. Manejo de valores nulos con 'N/A'
    print("Rellenando valores nulos en 'other_name', 'phone', 'email' con 'N/A'...")
    cursor.execute("""
        UPDATE employees
        SET other_name = 'N/A'
        WHERE other_name IS NULL;
    """)
    conexion.commit() # Confirmar los cambios

    cursor.execute("""
        UPDATE employees
        SET phone = 'N/A'
        WHERE phone IS NULL;
    """)
    conexion.commit() # Confirmar los cambios

    cursor.execute("""
        UPDATE employees
        SET email = 'N/A'
        WHERE email IS NULL;
    """)
    conexion.commit() # Confirmar los cambios
    print("Relleno de nulos completado.")

    # 2. Estandarización de capitalización para nombres
    # Aplicar CONCAT y SUBSTRING solo a los valores que NO sean 'N/A' o cadenas vacías
    print("Estandarizando capitalización de nombres...")

    # first_name
    cursor.execute("""
        UPDATE employees
        SET first_name = CONCAT(UPPER(SUBSTRING(first_name, 1, 1)), LOWER(SUBSTRING(first_name, 2)))
        WHERE first_name IS NOT NULL AND first_name != '';
    """)
    conexion.commit()

    # other_name (solo si no es 'N/A' o vacío)
    cursor.execute("""
        UPDATE employees
        SET other_name = CONCAT(UPPER(SUBSTRING(other_name, 1, 1)), LOWER(SUBSTRING(other_name, 2)))
        WHERE other_name IS NOT NULL AND other_name != 'N/A' AND other_name != '';
    """)
    conexion.commit()

    # first_surname
    cursor.execute("""
        UPDATE employees
        SET first_surname = CONCAT(UPPER(SUBSTRING(first_surname, 1, 1)), LOWER(SUBSTRING(first_surname, 2)))
        WHERE first_surname IS NOT NULL AND first_surname != '';
    """)
    conexion.commit()

    # second_surname (solo si no es 'N/A' o vacío)
    cursor.execute("""
        UPDATE employees
        SET second_surname = CONCAT(UPPER(SUBSTRING(second_surname, 1, 1)), LOWER(SUBSTRING(second_surname, 2)))
        WHERE second_surname IS NOT NULL AND second_surname != 'N/A' AND second_surname != '';
    """)
    conexion.commit()

    # full_name (solo si no es 'N/A' o vacío)
    cursor.execute("""
        UPDATE employees
        SET full_name = CONCAT(UPPER(SUBSTRING(full_name, 1, 1)), LOWER(SUBSTRING(full_name, 2)))
        WHERE full_name IS NOT NULL AND full_name != 'N/A' AND full_name != '';
    """)
    conexion.commit()

    print("Estandarización de capitalización completada.")

    print("\nLimpieza y transformación inicial de 'employees' finalizada exitosamente.")

except Exception as e:
    print(f"Ocurrió un error inesperado durante la limpieza y transformación inicial de 'employees': {e}")
    # En caso de error, intentar limpiar el cursor y revertir.
    try:
        # Asegurar que cualquier resultado pendiente sea consumido antes de rollback
        if cursor.description is not None:
            cursor.fetchall()
        conexion.rollback()
        print("Transacción revertida debido a un error.")
    except Exception as rollback_e:
        print(f"Error adicional al intentar revertir la transacción o limpiar el cursor: {rollback_e}")

✅ Conexión exitosa a la base de datos

--- Bloque 5: Limpieza y Transformación Inicial de 'employees' ---
Iniciando operaciones de limpieza en 'employees'...
Rellenando valores nulos en 'other_name', 'phone', 'email' con 'N/A'...
Relleno de nulos completado.
Estandarizando capitalización de nombres...
Estandarización de capitalización completada.

Limpieza y transformación inicial de 'employees' finalizada exitosamente.


In [16]:
import pandas as pd
import json
import numpy as np
from db_connection import get_engine, get_connection

print("--- Bloque 6: Limpieza y Aplanamiento de JSONs en 'employees' ---")

conexion = None
cursor = None
try:
    conexion = get_connection()
    cursor = conexion.cursor()
    engine = get_engine()
    print("✅ Conexión a la base de datos establecida y motor de Pandas configurado.")

    # Deshabilitar temporalmente las verificaciones de claves foráneas
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")

    # Modificar la estructura de las columnas JSON a VARCHAR(255) NULL
    alter_table_sql = """
        ALTER TABLE employees
        MODIFY COLUMN identity_document_type VARCHAR(255) NULL,
        MODIFY COLUMN country VARCHAR(255) NULL,
        MODIFY COLUMN region VARCHAR(255) NULL,
        MODIFY COLUMN city VARCHAR(255) NULL,
        MODIFY COLUMN emergency_contact VARCHAR(255) NULL,
        MODIFY COLUMN payment_method VARCHAR(255) NULL,
        MODIFY COLUMN bank VARCHAR(255) NULL,
        MODIFY COLUMN bank_account_type VARCHAR(255) NULL;
    """
    try:
        cursor.execute(alter_table_sql)
        conexion.commit()
    except Exception as e:
        conexion.rollback()
        print(f"❌ Error al modificar las columnas JSON: {e}")
        raise

    # Re-habilitar las verificaciones de claves foráneas
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")

    # Cargar la tabla 'employees' completa en un DataFrame de Pandas
    df_employees = pd.read_sql("SELECT * FROM employees;", engine)

    # Función para extraer valores de JSON (optimizado para tipos description, name, code)
    def extract_json_value(json_str):
        if pd.isna(json_str) or not isinstance(json_str, str) or not json_str.strip():
            return None
        if not json_str.strip().startswith('{') and not json_str.strip().startswith('['):
            return json_str # Ya aplanado o no es JSON

        try:
            data = json.loads(json_str)
            if isinstance(data, dict):
                for key in ['description', 'name', 'code']: # Orden de preferencia
                    if key in data and data[key] is not None:
                        return str(data[key])
            return None
        except json.JSONDecodeError:
            return json_str if json_str.strip() else None
        except Exception:
            return None

    # Columnas a aplanar
    json_columns = [
        'identity_document_type', 'country', 'region', 'city',
        'emergency_contact', 'payment_method', 'bank', 'bank_account_type'
    ]

    # Aplicar aplanamiento
    for col in json_columns:
        if col in df_employees.columns:
            df_employees[col] = df_employees[col].astype(str).apply(extract_json_value)

    # Preparar DataFrame para la actualización (manejo de nulos y tipos)
    cursor.execute("DESCRIBE `employees`;")
    db_columns_info = cursor.fetchall()
    db_column_names = [col_info[0] for col_info in db_columns_info]
    db_column_types = {col_info[0]: col_info[1] for col_info in db_columns_info}

    valid_columns_for_update = [
        col for col in df_employees.columns
        if isinstance(col, str) and col != 'id' and col in db_column_names
    ]

    for col in df_employees.columns:
        if col in valid_columns_for_update or col == 'id':
            if df_employees[col].dtype == 'object':
                df_employees[col] = df_employees[col].replace({np.nan: None, 'nan': None, '': None})
            elif pd.api.types.is_numeric_dtype(df_employees[col]) and df_employees[col].isnull().any():
                if pd.api.types.is_integer_dtype(df_employees[col]):
                    df_employees[col] = df_employees[col].astype(pd.Int64Dtype())
                df_employees[col] = df_employees[col].replace({np.nan: None})
            if pd.api.types.is_bool_dtype(df_employees[col]):
                df_employees[col] = df_employees[col].map({True: 1, False: 0, None: None, np.nan: None})

    # Actualizar los registros en la base de datos
    set_clauses = [f"`{col}` = %s" for col in valid_columns_for_update]
    update_sql = f"UPDATE `employees` SET {', '.join(set_clauses)} WHERE `id` = %s;"

    updated_count = 0
    errors_encountered = 0
    for index, row in df_employees.iterrows():
        update_values = [row[col] for col in valid_columns_for_update]
        update_values.append(row['id'])

        try:
            cursor.execute(update_sql, tuple(update_values))
            updated_count += 1
        except Exception as e:
            errors_encountered += 1
            print(f"❌ Error al actualizar ID {row['id']}: {e}")
            raise # Re-lanza el error para detener el script si falla un update crítico

    if errors_encountered > 0:
        conexion.rollback()
        print(f"❌ Actualización finalizada con {errors_encountered} errores. Transacción revertida.")
    else:
        conexion.commit()
        print(f"✅ Actualización de {updated_count} registros en la tabla 'employees' completada.")

    print("✅ Limpieza y aplanamiento de JSONs en 'employees' finalizada exitosamente.")

except Exception as e:
    print(f"❌ Ocurrió un error general: {e}")
    try:
        if conexion and conexion.is_connected():
            conexion.rollback()
    except Exception as rollback_e:
        print(f"❌ Error al revertir la transacción: {rollback_e}")
finally:
    if cursor:
        try:
            cursor.execute("SET FOREIGN_KEY_CHECKS = 1;") # Asegurar que las FKs se re-habilitan
        except Exception:
            pass # No imprimir si hay error al cerrar
    if conexion:
        pass # La conexión se mantiene abierta según tu solicitud

--- Bloque 6: Limpieza y Aplanamiento de JSONs en 'employees' ---
✅ Conexión exitosa a la base de datos
✅ Conexión a la base de datos establecida y motor de Pandas configurado.
✅ Actualización de 38 registros en la tabla 'employees' completada.
✅ Limpieza y aplanamiento de JSONs en 'employees' finalizada exitosamente.


In [17]:
import pandas as pd
import json
import numpy as np
from db_connection import get_engine, get_connection # Necesitamos get_connection para el cursor

print("--- Bloque 6: Limpieza y Aplanamiento de JSONs en 'employees' ---")

# --- PASO 1: Procesar y Aplanar en Pandas ---
try:
    engine = get_engine()
    df_employees = pd.read_sql("SELECT * FROM employees;", engine)
    print(f"DataFrame de 'employees' cargado. Registros: {len(df_employees)}")

    json_columns = [
        'identity_document_type', 'country', 'region', 'city',
        'emergency_contact', 'payment_method', 'bank', 'bank_account_type'
    ]

    def extract_json_value(json_str):
        if pd.isna(json_str) or not isinstance(json_str, str) or not json_str.strip():
            return None
        if not json_str.strip().startswith('{') and not json_str.strip().startswith('['):
            return json_str

        try:
            data = json.loads(json_str)
            if isinstance(data, dict):
                for key in ['description', 'name', 'code']:
                    if key in data and data[key] is not None:
                        return str(data[key])
            return None
        except json.JSONDecodeError:
            return json_str
        except Exception:
            return None

    for col in json_columns:
        if col in df_employees.columns:
            df_employees[col] = df_employees[col].astype(str).apply(extract_json_value)
    print("✅ Aplanamiento de columnas en DataFrame completado.")

    # Convertir 'None' a numpy.nan para que Pandas los maneje correctamente al guardar
    # Esto es crucial para que SQLAlchemy interprete None como NULL en la DB
    for col in df_employees.columns:
        df_employees[col] = df_employees[col].replace({None: np.nan})


    # --- PASO 2: Guardar en una tabla temporal y realizar UPSERT ---
    temp_table_name = 'temp_employees_cleaned' # Nombre de la tabla temporal

    # 1. Guardar el DataFrame limpio en una tabla temporal (esto es seguro, ya que es una tabla nueva)
    print(f"Guardando el DataFrame limpio en la tabla temporal '{temp_table_name}'...")
    df_employees.to_sql(name=temp_table_name, con=engine, if_exists='replace', index=False)
    print(f"✅ DataFrame guardado en '{temp_table_name}' exitosamente.")

    # 2. Realizar el UPSERT (UPDATE + INSERT) desde la tabla temporal a la tabla original
    conexion = get_connection()
    cursor = conexion.cursor()

    # --- CORRECCIÓN AQUÍ: Las columnas a actualizar deben usar VALUES() o el alias 's' ---
    # `VALUES(col_name)` se refiere al valor que sería INSERTADO
    # Alternativamente, puedes usar `s.col_name` si `s` es el alias de la tabla temporal en el SELECT subyacente.
    # Dado que `s` ya se usa, la opción `s.col_name` es más directa.
    update_cols = [col for col in df_employees.columns if col != 'id']
    set_clauses = ", ".join([f"`{col}` = s.`{col}`" for col in update_cols]) # <-- CORRECCIÓN: usamos s.`col`

    insert_cols = ", ".join([f"`{col}`" for col in df_employees.columns])
    insert_values = ", ".join([f"s.`{col}`" for col in df_employees.columns])

    upsert_sql = f"""
    INSERT INTO employees ({insert_cols})
    SELECT {insert_values}
    FROM {temp_table_name} s
    ON DUPLICATE KEY UPDATE {set_clauses};
    """
    print("Ejecutando UPSERT de datos limpios a la tabla 'employees'...")
    cursor.execute(upsert_sql)
    conexion.commit()
    print("✅ UPSERT completado: Datos de 'employees' actualizados/insertados exitosamente.")

    # Opcional: Eliminar la tabla temporal después del UPSERT
    print(f"Eliminando tabla temporal '{temp_table_name}'...")
    cursor.execute(f"DROP TABLE {temp_table_name};")
    conexion.commit()
    print(f"✅ Tabla temporal '{temp_table_name}' eliminada.")

    print("\n✅ Limpieza y aplanamiento de JSONs en 'employees' finalizada exitosamente.")

except Exception as e:
    print(f"❌ Ocurrió un error general durante la limpieza y aplanamiento de JSONs en 'employees': {e}")
    print("Por favor, revisa la conexión a la base de datos, los permisos o el nombre de la tabla.")
    if 'conexion' in locals() and conexion and conexion.is_connected():
        conexion.rollback()
finally:
    if 'cursor' in locals() and cursor:
        cursor.close()
    if 'conexion' in locals() and conexion:
        # Se mantiene la conexión abierta si así lo necesitas, pero se cierra el cursor.
        pass # Mantener la conexión abierta

--- Bloque 6: Limpieza y Aplanamiento de JSONs en 'employees' ---
DataFrame de 'employees' cargado. Registros: 38
✅ Aplanamiento de columnas en DataFrame completado.
Guardando el DataFrame limpio en la tabla temporal 'temp_employees_cleaned'...


C:\Users\ALEXANDRA\AppData\Local\Temp\ipykernel_2540\3978390001.py:45: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_employees[col] = df_employees[col].replace({None: np.nan})


✅ DataFrame guardado en 'temp_employees_cleaned' exitosamente.
✅ Conexión exitosa a la base de datos
Ejecutando UPSERT de datos limpios a la tabla 'employees'...
✅ UPSERT completado: Datos de 'employees' actualizados/insertados exitosamente.
Eliminando tabla temporal 'temp_employees_cleaned'...
✅ Tabla temporal 'temp_employees_cleaned' eliminada.

✅ Limpieza y aplanamiento de JSONs en 'employees' finalizada exitosamente.


*TABLA 3: employee_contracts o Contrato de empleados*


In [18]:
import pandas as pd
from db_connection import get_engine, get_connection 

print("--- Bloque 7: Exploración de 'employee_contracts' ---")

conexion = None
cursor = None
try:
    # Reestablecer la conexión y el cursor
    conexion = get_connection()
    cursor = conexion.cursor()
    engine = get_engine() # Obtener el engine para pandas

    # Cargar una pequeña muestra de la tabla para inspeccionar columnas y datos
    df_contracts_sample = pd.read_sql("SELECT * FROM employee_contracts LIMIT 5;", engine)

    print("Columnas de la tabla 'employee_contracts':")
    print(df_contracts_sample.columns.tolist()) # Lista concisa de nombres de columnas

    print("\nPrimeras 5 filas de datos de 'employee_contracts':")
    print(df_contracts_sample.to_string()) # Usar to_string() para evitar truncamiento

except Exception as e:
    print(f"❌ Ocurrió un error al explorar 'employee_contracts': {e}")
finally:
    # Cerrar el cursor y la conexión
    if cursor:
        cursor.close()
    if conexion:
        conexion.close()
    print("✅ Conexión cerrada al finalizar el bloque.")

--- Bloque 7: Exploración de 'employee_contracts' ---
✅ Conexión exitosa a la base de datos
Columnas de la tabla 'employee_contracts':
['id', 'employee_id', 'employee_type', 'employee_subtype', 'high_penson_risk', 'integral_salary', 'type', 'start_date', 'end_date', 'salary', 'transport_subsidy', 'risk_type', 'holidays_enjoyed', 'health_provider_id', 'health_provider', 'pension_provider', 'pension_provider_id', 'severance_provider', 'severance_provider_id', 'family_compensation_fund_provider', 'family_compensation_fund_provider_id', 'settlement_cause', 'settlement_with_just_cause', 'settlement_probationary_period', 'settlement_date', 'settlement_worked_days', 'settlement_severance_base_days', 'settlement_service_bonus_base_days', 'settlement_holidays_to_pay', 'settlement_salary_base', 'settlement_holidays_base', 'settlement_severance_base', 'settlement_service_bonus_base', 'settlement_non_salary_income', 'settlement_loans', 'settlement_deductions', 'settlement_holidays', 'settlement_se

📜 *Análisis Detallado de la Tabla employee_contracts*

El Corazón de la Relación Laboral

Esta tabla es el historial de contratos laborales, un registro vital de todos los acuerdos de tus empleados, desde los términos de empleo hasta sus salarios y proveedores de beneficios. Es fundamental para una gestión de RRHH robusta y para entender el ciclo de vida del talento.

🔍 Los Datos Que Importan

id & employee_id: Son la identificación única del contrato y su vínculo directo con cada empleado.
employee_type, type: Te dicen qué tipo de contrato (ej. "fijo", "indefinido") y rol tiene el empleado.
start_date, end_date: Marcan el periodo de vigencia del contrato, clave para antigüedad.
salary: El monto de compensación acordado.
risk_type: Informa sobre el riesgo laboral asociado al puesto.
health_provider, pension_provider, severance_provider: Datos esenciales de los proveedores de seguridad social.
settlement_approved_at: Indica cuándo fue liquidado un contrato.
🛠️ Nuestro Enfoque de Limpieza
La magia está en transformar estos datos para que brillen:

Aplanar JSONs: Columnas como employee_type, type, risk_type y todas las de _provider vienen en formato JSON. Las "aplanaremos" para extraer su valor más útil (como la descripción o el nombre) y convertirlas en texto plano.
Fechas Especiales: end_date y settlement_approved_at pueden ser nulas si el contrato sigue activo o no ha sido liquidado. Es crucial manejar estos NULLs inteligentemente para cálculos precisos (ej. duración del contrato).
Integridad de Enlaces: Nos aseguraremos de que cada employee_id tenga su contraparte en la tabla de employees para evitar datos huérfanos.
📈 Insights para tu Dashboard de RRHH
Con esta tabla limpia, podrás responder preguntas clave como:

Tipos de Contratos: ¿Cuál es la distribución de contratos (fijos, indefinidos, por obra)?
Duración y Rotación: ¿Cuál es la duración promedio de nuestros contratos y cuáles son las tendencias de finalización?
Salarios vs. Contratos: ¿Cómo se correlaciona el salario con el tipo de contrato?
Proveedores Clave: ¿Cuáles son los proveedores de salud, pensión y cesantías más usados por nuestros empleados?



In [19]:
import pandas as pd
from db_connection import get_connection # Asegúrate de tener get_connection configurado

print("--- Bloque 8: Verificación Inicial de 'employee_contracts' ---")

conexion = None
cursor = None
try:
    # Reestablecer la conexión y el cursor
    conexion = get_connection()
    cursor = conexion.cursor()
    print("✅ Conexión a la base de datos establecida.")

    # 1. Obtener el número total de registros
    cursor.execute("SELECT COUNT(*) FROM employee_contracts;")
    total_registros_contracts = cursor.fetchone()[0]
    print(f"\nTotal de registros en 'employee_contracts': {total_registros_contracts}")

    # 2. Verificar valores nulos por columna (solo se muestran si hay nulos)
    print("\nValores nulos en 'employee_contracts':")
    cursor.execute("DESCRIBE employee_contracts;")
    columnas_info = cursor.fetchall()

    nulls_found = False
    for col_info in columnas_info:
        nombre_columna = col_info[0]
        cursor.execute(f"SELECT COUNT(*) FROM employee_contracts WHERE `{nombre_columna}` IS NULL;")
        nulos_count = cursor.fetchone()[0]
        if nulos_count > 0:
            print(f"  - `{nombre_columna}`: {nulos_count} nulos")
            nulls_found = True
    if not nulls_found:
        print("  ✅ No se encontraron valores nulos en ninguna columna.")

    # 3. Verificar filas duplicadas
    cursor.execute("SELECT COUNT(id) - COUNT(DISTINCT id) FROM employee_contracts;")
    duplicados_id_contracts = cursor.fetchone()[0]
    print(f"\nFilas duplicadas basadas en 'id': {duplicados_id_contracts}")

    cursor.execute("""
        SELECT COUNT(*)
        FROM (
            SELECT employee_id, start_date, COUNT(*)
            FROM employee_contracts
            GROUP BY employee_id, start_date
            HAVING COUNT(*) > 1
        ) AS duplicated_contracts;
    """)
    duplicados_employee_date = cursor.fetchone()[0]
    print(f"Contratos duplicados (employee_id + start_date): {duplicados_employee_date}")

    # 4. Obtener valores únicos para columnas booleanas (tinyint)
    print("\nValores únicos en columnas booleanas:")
    boolean_columns = ['high_penson_risk', 'integral_salary', 'transport_subsidy']
    for col in boolean_columns:
        cursor.execute(f"SELECT DISTINCT `{col}` FROM employee_contracts;")
        unicos = [fila[0] for fila in cursor.fetchall()]
        print(f"  - `{col}`: {unicos}")

    # 5. Muestra de contenido de una columna previamente JSON (ahora aplanada): 'employee_type' (primeros 5)
    print("\nMuestra de valores de 'employee_type' (primeros 5):")
    # Selecciona valores NO NULOS y los que NO sean 'None' en formato string si ya fueron aplanados.
    cursor.execute("SELECT employee_type FROM employee_contracts WHERE employee_type IS NOT NULL AND employee_type != 'None' LIMIT 5;")
    employee_type_sample = [fila[0] for fila in cursor.fetchall()]
    if employee_type_sample:
        for i, etype in enumerate(employee_type_sample):
            print(f"  - Muestra {i+1}: '{etype}'")
    else:
        print("  (No hay valores no nulos ni 'None' para mostrar en 'employee_type' o la columna ya está completamente limpia/vacía).")


except Exception as e:
    print(f"❌ Ocurrió un error durante la verificación inicial de datos en 'employee_contracts': {e}")
finally:
    if cursor:
        cursor.close()
    if conexion:
        conexion.close()
    print("✅ Conexión cerrada al finalizar el bloque.")

--- Bloque 8: Verificación Inicial de 'employee_contracts' ---
✅ Conexión exitosa a la base de datos
✅ Conexión a la base de datos establecida.

Total de registros en 'employee_contracts': 43

Valores nulos en 'employee_contracts':
  - `employee_type`: 43 nulos
  - `employee_subtype`: 43 nulos
  - `type`: 43 nulos
  - `end_date`: 32 nulos
  - `risk_type`: 43 nulos
  - `health_provider_id`: 5 nulos
  - `health_provider`: 43 nulos
  - `pension_provider`: 43 nulos
  - `pension_provider_id`: 5 nulos
  - `severance_provider`: 43 nulos
  - `severance_provider_id`: 5 nulos
  - `family_compensation_fund_provider_id`: 5 nulos
  - `settlement_cause`: 30 nulos
  - `settlement_date`: 30 nulos
  - `settlement_worked_days`: 30 nulos
  - `settlement_holidays_to_pay`: 30 nulos
  - `settlement_salary_base`: 30 nulos
  - `settlement_holidays_base`: 30 nulos
  - `settlement_severance_base`: 30 nulos
  - `settlement_service_bonus_base`: 30 nulos
  - `settlement_non_salary_income`: 30 nulos
  - `settlement

In [20]:
import pandas as pd
import json
import numpy as np
from db_connection import get_engine, get_connection 

print("--- Bloque 9: Limpieza y Transformación de 'employee_contracts' ---")

conexion = None 
cursor = None 
try:
    # Reestablecemos conexión a la base de datos para asegurar un estado limpio
    conexion = get_connection()
    cursor = conexion.cursor()
    engine = get_engine() 
    print("✅ Conexión y motor de base de datos listos.")

    # --- PASO 1: Cargando y Transformando en Pandas ---
    print("\n--- PASO 1: Transformaciones en Pandas ---")
    df_contracts = pd.read_sql("SELECT * FROM employee_contracts;", engine)
    print(f"DataFrame de contratos cargado. Registros: {len(df_contracts)}")

    # 1.1. Manejo de valores nulos y asegurando nombres de columnas válidos
    # Rellena valores nulos específicos para columnas de texto e IDs numéricos
    cols_to_fill_na = [
        'health_provider_id', 
        'pension_provider',
        'pension_provider_id',
        'severance_provider',
        'severance_provider_id'
    ]
    for col in cols_to_fill_na:
        if col in df_contracts.columns:
            if 'id' not in col: # Columnas de texto
                df_contracts[col] = df_contracts[col].astype(str).replace('nan', 'N/A').fillna('N/A')
            else: # Columnas de ID numérico
                df_contracts[col] = df_contracts[col].replace({np.nan: None})
    
    # Asegura que los nombres de las columnas sean strings válidos
    df_contracts.columns = [str(col) if pd.notna(col) else f"unnamed_col_{i}" for i, col in enumerate(df_contracts.columns)]
    print("✅ Relleno de nulos y validación de nombres de columnas completados.")

    # 1.2. Aplanamiento de Columnas JSON
    json_columns = [
        'employee_type', 'employee_subtype', 'type', 'risk_type',
        'health_provider', 'pension_provider', 'severance_provider'
        # 'family_compensation_fund_provider' sigue excluida
    ]

    def extract_json_value(json_str):
        if pd.isna(json_str) or not isinstance(json_str, str) or not json_str.strip().startswith('{'):
            return None 
        try:
            data = json.loads(json_str)
            if isinstance(data, dict):
                # Priorizamos 'description', luego 'name', luego 'code'
                if 'description' in data and data['description'] is not None:
                    return str(data['description'])
                elif 'name' in data and data['name'] is not None:
                    return str(data['name'])
                elif 'code' in data and data['code'] is not None: 
                    return str(data['code'])
            return None 
        except json.JSONDecodeError:
            return None 
        except Exception:
            return None

    print("\nIniciando aplanamiento de columnas JSON...")
    for col in json_columns:
        if col in df_contracts.columns:
            df_contracts[col] = df_contracts[col].astype(str).apply(extract_json_value)
            # print(f"    Columna '{col}' aplanada.") # Eliminar prints de cada columna
        # else:
            # print(f"    Advertencia: Columna '{col}' no encontrada para aplanar JSON.") # Eliminar warnings si no están
    print("✅ Aplanamiento de JSONs en DataFrame completado.")
    
    # --- PASO 2: Actualizando los registros en la Base de Datos ---
    print("\n--- PASO 2: Actualizando la Base de Datos ---")
    
    # Obtenemos los nombres de las columnas de la DB para un UPDATE seguro
    cursor.execute("DESCRIBE `employee_contracts`;")
    db_column_names = [col_info[0] for col_info in cursor.fetchall()]

    # Filtramos las columnas del DataFrame que realmente existen en la DB y no son 'id'
    valid_columns_for_update = [
        col for col in df_contracts.columns
        if isinstance(col, str) and col != 'id' and col in db_column_names
    ]
    
    if not valid_columns_for_update:
        print("⚠️ Advertencia: No hay columnas válidas para actualizar (aparte del 'id'). No se realizará ninguna actualización.")
    else:
        # Construimos la sentencia UPDATE dinámicamente
        set_clauses = [f"`{col}` = %s" for col in valid_columns_for_update]
        update_sql = f"UPDATE `employee_contracts` SET {', '.join(set_clauses)} WHERE `id` = %s;"
        
        updated_count = 0
        total_rows = len(df_contracts)
        
        # Iteramos y actualizamos fila por fila
        for index, row in df_contracts.iterrows():
            update_values = []
            for col in valid_columns_for_update:
                val = row[col]
                # Convertimos valores de Pandas/NumPy a tipos compatibles con MySQL
                if pd.isna(val) or (isinstance(val, (float, np.float64)) and np.isnan(val)):
                    update_values.append(None) 
                elif isinstance(val, (np.int64, np.int32)):
                    update_values.append(int(val)) 
                elif isinstance(val, (np.bool_)):
                    update_values.append(bool(val)) 
                else:
                    update_values.append(val)
            update_values.append(row['id']) 
            
            try:
                cursor.execute(update_sql, tuple(update_values))
                updated_count += 1
                if updated_count % 50 == 0 or updated_count == total_rows: # Progreso cada 50 o al final
                    print(f"    Progreso: {updated_count}/{total_rows} registros actualizados...")
            except Exception as e:
                print(f"❌ Error al actualizar el registro con ID {row['id']}: {e}")
                conexion.rollback()
                raise 

        conexion.commit() 
        print(f"✅ Actualización de {updated_count} registros en la tabla 'employee_contracts' completada exitosamente.")
    
    print("\n✅ Limpieza y transformación de 'employee_contracts' finalizada exitosamente.")

except Exception as e:
    print(f"❌ Ocurrió un error general durante la limpieza de 'employee_contracts': {e}")
    try:
        if 'conexion' in locals() and conexion and conexion.is_connected():
            conexion.rollback()
            print("Transacción revertida debido a un error.")
    except Exception as rollback_e:
        print(f"❌ Error adicional al intentar revertir: {rollback_e}")
finally:
    if cursor:
        cursor.close()
    if conexion:
        conexion.close()
    print("✅ Conexión a la base de datos cerrada.")

--- Bloque 9: Limpieza y Transformación de 'employee_contracts' ---
✅ Conexión exitosa a la base de datos
✅ Conexión y motor de base de datos listos.

--- PASO 1: Transformaciones en Pandas ---
DataFrame de contratos cargado. Registros: 43
✅ Relleno de nulos y validación de nombres de columnas completados.

Iniciando aplanamiento de columnas JSON...
✅ Aplanamiento de JSONs en DataFrame completado.

--- PASO 2: Actualizando la Base de Datos ---
    Progreso: 43/43 registros actualizados...
✅ Actualización de 43 registros en la tabla 'employee_contracts' completada exitosamente.

✅ Limpieza y transformación de 'employee_contracts' finalizada exitosamente.
✅ Conexión a la base de datos cerrada.


In [21]:
# Cerrar conexión
cursor.close()
conexion.close()


In [22]:
import pandas as pd
import plotly.express as px

In [23]:
# data_utils.py
import pandas as pd
import json

def extract_or_normalize_contract_type(value):
    """
    Normaliza un valor que puede ser JSON, nulo o una cadena simple a un tipo de contrato.
    Retorna 'No Definido' si no se puede extraer un tipo válido.
    """
    if pd.isna(value) or not isinstance(value, str) or not str(value).strip():
        return 'No Definido'

    cleaned_value = str(value).strip()

    # Si la cadena NO parece un JSON, la retorna directamente (asume que ya está aplanada)
    if not cleaned_value.startswith('{') and not cleaned_value.startswith('['):
        return cleaned_value

    # Intentar parsear JSON
    try:
        data = json.loads(cleaned_value)
        if isinstance(data, dict):
            # Prioridad: 'description' -> 'name' -> 'code'
            for key in ['description', 'name', 'code']:
                if key in data and data[key] is not None:
                    return str(data[key]).strip()
        return 'No Definido' # JSON válido pero sin las claves esperadas
    except json.JSONDecodeError:
        return cleaned_value # No es JSON válido, usar el valor crudo
    except Exception:
        return 'No Definido' # Cualquier otro error

In [24]:
%%writefile data_utils.py
import pandas as pd
import json

def extract_or_normalize_contract_type(value):
    """
    Normaliza un valor que puede ser JSON, nulo o una cadena simple a un tipo de contrato.
    Retorna 'No Definido' si no se puede extraer un tipo válido.
    """
    if pd.isna(value) or not isinstance(value, str) or not str(value).strip():
        return 'No Definido'

    cleaned_value = str(value).strip()

    # Si la cadena NO parece un JSON, la retorna directamente (asume que ya está aplanada)
    if not cleaned_value.startswith('{') and not cleaned_value.startswith('['):
        return cleaned_value

    # Intentar parsear JSON
    try:
        data = json.loads(cleaned_value)
        if isinstance(data, dict):
            # Prioridad: 'description' -> 'name' -> 'code'
            for key in ['description', 'name', 'code']:
                if key in data and data[key] is not None:
                    return str(data[key]).strip()
        return 'No Definido' # JSON válido pero sin las claves esperadas
    except json.JSONDecodeError:
        return cleaned_value # No es JSON válido, usar el valor crudo
    except Exception:
        return 'No Definido' # Cualquier otro error

Overwriting data_utils.py


In [25]:
import pandas as pd
from db_connection import get_engine, get_connection

conexion = None
try:
    engine = get_engine()
    conexion = get_connection()
    cursor = conexion.cursor()

    print("\nListando tablas en la base de datos 'novateam'...")
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()
    print("Tablas encontradas:")
    for table in tables:
        print(f"- {table[0]}") # table[0] contiene el nombre de la tabla
    print("✅ Listado de tablas completado.")

except Exception as e:
    print(f"\n❌❌ Ocurrió un error al listar tablas: {e}")
finally:
    if conexion and conexion.is_connected():
        conexion.close()

✅ Conexión exitosa a la base de datos

Listando tablas en la base de datos 'novateam'...
Tablas encontradas:
- accounting_account_balances
- accounting_accounts
- accounting_movements
- accounting_voucher_items
- accounting_voucher_types
- accounting_vouchers
- billing_numberings
- contact_relationships
- contact_statements
- contacts
- contract_salary_history
- costs_and_expenses
- costs_and_expenses_categories
- coupons
- document_items
- documents
- ecommerce_legal_info
- ecommerce_purchase_orders
- employee_contracts
- employee_positions
- employees
- fixed_asset_depreciations
- fixed_assets
- fixed_assets_groups
- headquarter_warehouses
- headquarters
- inventory_adjustments
- inventory_groups
- item_balance
- item_categories
- item_depreciations
- item_kardex
- item_subcategories
- item_variations
- items
- payment_conditions
- payments
- payroll_deductions
- payroll_details
- payroll_incomes
- payroll_providers
- payrolls
- price_lists
- retention_concepts
- retentions
- retenti

---




📊 *#Análisis Detallado de Tablas Clave para Gestión de Recursos Humanos# (RRHH)*
Este documento presenta un resumen conciso y visual de las tablas más relevantes para el análisis de RRHH, destacando su propósito, contenido clave y las consideraciones para futuras extracciones, transformaciones y cargas (ETL), así como para la construcción de Indicadores Clave de Rendimiento (KPIs).

👥 *1. employees (Empleados)*✨

✨ El Corazón de la Información del Personal 

Esta tabla es la fuente principal de datos demográficos y de contacto de cada individuo en la organización. Esencial para la segmentación y el entendimiento de la fuerza laboral.

🔑 *Aspectos Clave para el Análisis de RRHH:*
Identificación Única: id, uuid, document_number, full_name. Permiten rastrear y referenciar a cada empleado.
Datos Demográficos: birth_date (para grupos de edad), country, region, city, address. Cruciales para análisis de diversidad, localización y planificación estratégica.
Contacto y Comunicación: phone, email.
Estructura Organizacional: headquarter_id (sede), company_area_id (área/departamento), employee_position_id (cargo). Permiten análisis de personal por ubicación y estructura interna.
Estado Actual: status (activo/inactivo). Fundamental para determinar la plantilla activa y el cálculo de la rotación.
Fechas Clave: created_at (fecha de ingreso a la empresa) y updated_at. created_at es vital para el cálculo de la antigüedad.
🛠️ *Limpieza y Transformación Realizada:*
Aplanamiento de JSONs: Columnas como identity_document_type, country, region, city, emergency_contact, payment_method, bank, bank_account_type fueron transformadas de JSON a valores planos (VARCHAR). Esto facilita su uso directo en consultas y análisis.
Manejo de Valores Nulos: Se rellenaron nulos en other_name, phone, email con 'N/A' para asegurar la completitud de los registros.
Estandarización de Texto: Se normalizó la capitalización de nombres y apellidos para evitar inconsistencias y mejorar la calidad de los datos textuales.
Consistencia de Datos: Los datos limpios se actualizaron/insertaron en la tabla employees (UPSERT), garantizando que los análisis futuros usen la versión más limpia.
💡 Consideraciones para KPIs:
Antigüedad del Empleado: Calcular con created_at (o la fecha de inicio del primer contrato, start_date, si es anterior).
Composición de la Fuerza Laboral: Distribución por grupos de edad, tipo de documento, ubicación geográfica.
Empleados Activos vs. Inactivos: Monitoreo del status para el conteo de la plantilla.


💼 2. *employee_positions (Cargos / Posiciones de Empleados)*

✨ Definiendo la Estructura de Roles de la Organización ✨

Esta tabla cataloga los distintos roles o cargos que existen dentro de la empresa, proporcionando una base para entender la estructura jerárquica y de compensación.

🔑 Aspectos Clave para el Análisis de RRHH:
Identificación de Posición: id (identificador único), name (nombre del puesto, ej., "Gerente de Proyectos", "Desarrollador Senior").
Detalle del Rol: description.
Referencia Salarial: salary (salario asociado a la posición). Esencial para análisis de estructura salarial y comparaciones.
Estado del Puesto: status (activo/inactivo).
🛠️ Limpieza y Transformación Realizada:
Normalización de Nombres: Corrección de errores tipográficos comunes (ej., 'Secretria' a 'Secretaria') en la columna name para asegurar la uniformidad.
Manejo de Valores Nulos: Relleno de nulos en description con 'N/A' para garantizar la consistencia en los metadatos.
💡 Consideraciones para KPIs:
Salario Promedio por Cargo: Unir con employees y employee_contracts para obtener el salario real pagado por cada posición.
Conteo de Empleados por Posición: Relacionar con employees para saber cuántos colaboradores ocupan cada puesto.
Antigüedad por Cargo: Calcular la antigüedad promedio de los empleados dentro de cada posición.

📜 *3. employee_contracts (Contratos de Empleados)*

✨ El Historial de la Relación Laboral y sus Términos ✨

Esta tabla es el registro fundamental de todos los acuerdos contractuales con los empleados, incluyendo términos, salarios específicos y detalles de beneficios. Es crítica para la gestión del ciclo de vida del talento.

🔑 *Aspectos Clave para el Análisis de RRHH:*
Vínculo con Empleado: employee_id (clave foránea a employees).
Fechas Contractuales: start_date (inicio del contrato) y end_date (fin del contrato). Vital para calcular la antigüedad precisa y la rotación.
Compensación Específica: salary. Este es el salario individual acordado, que es la base para los cálculos de nómina.
Proveedores de Beneficios: health_provider_id, pension_provider_id, severance_provider_id, family_compensation_fund_provider (puede contener JSON).
Información de Desvinculación: status (activo/inactivo del contrato), settlement_cause, settlement_date.
🛠️ Limpieza y Transformación Realizada:
Aplanamiento de JSONs: Columnas como family_compensation_fund_provider fueron procesadas para extraer los datos útiles.
Manejo de Valores Nulos: Se identificó la presencia de nulos en varias columnas (end_date, employee_type, type, etc.). Es crucial interpretar end_date nulo como un contrato activo.
Actualización en la BD: Los datos limpios se actualizaron/insertaron, asegurando la consistencia de la información contractual.
💡 Consideraciones para KPIs:
Antigüedad de Empleados: Utilizar start_date (y end_date si aplica) para un cálculo preciso de la permanencia.
Tasa de Rotación: start_date, end_date y settlement_date son fundamentales para identificar entradas y salidas de personal.
Análisis Salarial Detallado: salary permite analizar la distribución de salarios reales y la equidad salarial.
Gestión de Beneficios: Los IDs de los proveedores permiten analizar la distribución de empleados por entidad de salud, pensión, etc.

📈 *4. contract_salary_history (Historial Salarial de Contratos)*

✨ Trazabilidad de la Evolución Salarial Individual ✨

Esta tabla, aunque no se sometió a un ETL completo, es crucial para entender cómo ha evolucionado el salario de cada empleado a lo largo de su trayectoria en la empresa.

🔑 Aspectos Clave Potenciales para el Análisis de RRHH:
Referencia Contractual: contract_id o employee_contract_id: Vincula cada registro de cambio salarial a un contrato específico.
Fecha de Efectividad: effective_date o change_date: La fecha a partir de la cual un nuevo salario se hizo efectivo. Es vital para análisis de tendencias.
Valores de Salario: old_salary, new_salary o salary_amount: Registra el monto del salario en ese punto en el tiempo.
Justificación del Cambio: reason (ej., "Aumento por Desempeño", "Ascenso", "Ajuste Salarial Anual").
💡 Consideraciones para Futuro ETL y KPIs:
Análisis de Incrementos Salariales: Calcular el porcentaje y la frecuencia de los aumentos salariales a lo largo del tiempo, por empleado o por grupo.
Determinación del Salario Actual Preciso: Para saber el salario de un empleado en una fecha específica, se buscaría el registro más reciente antes o en esa fecha.
Uniones Necesarias: Requiere uniones con employee_contracts y employees para contextualizar los cambios salariales por empleado, posición, etc.

💰 5. *payrolls (Nóminas Principales)*

✨ Visión Consolidada del Costo de la Fuerza Laboral por Período ✨

Esta tabla probablemente contiene los resúmenes y totales de cada ciclo de nómina, ofreciendo una perspectiva global del costo de la nómina para la empresa.

🔑 Aspectos Clave Potenciales para el Análisis de RRHH:
Identificación del Período: id, start_date, end_date (periodo que abarca la nómina), payment_date (fecha en que se realizó el pago).
Totales Monetarios: total_salary, total_deductions, total_incomes, total_payroll_cost (o net_payroll_amount). Estos son los montos agregados de todos los pagos y deducciones en ese período. (Confirmado en el KPI 1).
Estado del Proceso: status (ej., "Procesada", "Pagada", "Anulada").
Referencias Administrativas: user_id (quién procesó), headquarter_id (a qué sede corresponde esta nómina).
💡 Consideraciones para Futuro ETL y KPIs:
KPI de Costo Total de Nómina: Directamente utilizable para el KPI 1: Costo Total de Nómina por Período.
Tendencias Financieras de RRHH: Análisis de la evolución mensual o trimestral de los costos de nómina, salarios, ingresos y deducciones.
Integración de Datos: Para un análisis granular, necesitará uniones con payroll_details, payroll_incomes, y payroll_deductions.

📑 *6. payroll_details (Detalles de Nómina)*

✨ El Desglose Concepto por Concepto de Cada Pago de Nómina ✨

Esta tabla es el corazón de la granularidad de la nómina, mostrando cada componente de ingresos y deducciones para cada empleado en un período de pago.

🔑 Aspectos Clave Potenciales para el Análisis de RRHH:
Vínculo con Nómina y Empleado: payroll_id (conecta a payrolls), employee_id (conecta a employees).
Concepto de Pago: income_id (si es un ingreso) o deduction_id (si es una deducción). Alternativamente, un concept_id junto con un campo type (ej., 'ingreso', 'deducción').
Monto Detallado: amount: El valor monetario exacto de cada concepto para un empleado en un período.
Unidades de Tiempo: worked_days, hours_worked: Si la nómina se basa en el tiempo trabajado, estas columnas son cruciales.
💡 Consideraciones para Futuro ETL y KPIs:
Análisis Salarial Detallado: Permite construir informes sobre la composición del pago de los empleados (salario base, bonificaciones, horas extras, etc.).
Verificación de Nómina: Útil para auditar los pagos individuales y asegurar que suman los totales en payrolls.
Cálculo de Ingresos y Deducciones por Tipo: Unir con payroll_incomes y payroll_deductions para categorizar y totalizar montos específicos.

➕ *7. payroll_incomes (Ingresos de Nómina)*

✨ Clasificación de Conceptos que Aumentan el Pago ✨

Esta tabla define los distintos tipos de ingresos (adicionales al salario base) que los empleados pueden recibir.

🔑 Aspectos Clave Potenciales para el Análisis de RRHH:
Identificador de Concepto: id (único para cada tipo de ingreso).
Nombre del Ingreso: name (ej., "Horas Extras", "Bonificación por Desempeño", "Comisiones de Ventas").
Descripción Detallada: description.
Impacto Fiscal/Social: taxable (si es gravable), social_security_applicable (si está sujeto a seguridad social).
💡 Consideraciones para Futuro ETL y KPIs:
Análisis de la Composición de Ingresos: Permite desglosar y analizar qué tipos de ingresos son más comunes o representan una mayor proporción del pago total.
Impacto en Costos: Útil para entender los costos asociados a bonificaciones, horas extras, etc.
Cumplimiento: Las columnas taxable y social_security_applicable son importantes para asegurar la correcta clasificación en los reportes fiscales.

➖ *8. payroll_deductions (Deducciones de Nómina)*

✨ Clasificación de Conceptos que Reducen el Pago ✨

Similar a payroll_incomes, pero esta tabla define los diferentes tipos de deducciones aplicadas al salario de los empleados.

🔑 Aspectos Clave Potenciales para el Análisis de RRHH:
Identificador de Concepto: id (único para cada tipo de deducción).
Nombre de la Deducción: name (ej., "Aportes Salud", "Aportes Pensión", "Retención en la Fuente", "Préstamo Empresa").
Descripción Detallada: description.
Naturaleza de la Deducción: mandatory (si es obligatoria), is_loan (si corresponde a un pago de préstamo).
💡 Consideraciones para Futuro ETL y KPIs:
Análisis de Costos para Empleados: Permite entender cuánto del salario bruto se destina a deducciones obligatorias (seguridad social) y voluntarias (préstamos, fondos).
Gestión de Préstamos: is_loan es clave para el seguimiento de los préstamos concedidos a los empleados.
Cumplimiento: Las columnas de tipo (mandatory) son importantes para la auditoría y los reportes.

🏢 *9. payroll_providers (Proveedores de Nómina / Beneficios)*

✨ Gestión de Entidades Externas Relacionadas con RRHH ✨

Esta tabla enumera los proveedores externos (ej., EPS, Fondos de Pensiones, Cajas de Compensación) con los que la empresa interactúa para la gestión de nómina y beneficios.

🔑 Aspectos Clave Potenciales para el Análisis de RRHH:
Identificación del Proveedor: id, name (ej., "Sura EPS", "Porvenir", "Cafam").
Tipo de Servicio: type (ej., "Salud", "Pensión", "Cesantías", "Caja de Compensación Familiar").
Información de Contacto: contact_info.
💡 Consideraciones para Futuro ETL y KPIs:
Integración de Datos: Se uniría con employee_contracts (a través de los IDs de proveedor) y, potencialmente, payroll_details para analizar costos y distribución de empleados por proveedor de beneficio.
Rendimiento de Proveedores: Aunque no directamente en esta tabla, se puede usar para vincular el gasto total con cada proveedor.

💸 *10. costs_and_expenses (Costos y Gastos)*

✨ Gastos Operativos Directamente Relacionados con el Personal ✨

Aunque es una tabla de naturaleza financiera general, su inclusión en el ámbito de RRHH implica que puede contener registros de gastos asociados directamente al personal, fuera de la nómina tradicional.

🔑 Aspectos Clave Potenciales para el Análisis de RRHH:
Detalle del Gasto: id (identificador), date (fecha en que incurrió el gasto), amount (monto monetario).
Categorización: category_id (clave foránea a costs_and_expenses_categories para clasificar el tipo de gasto).
Vínculo con Empleado (Opcional): employee_id (si un gasto específico puede ser atribuido a un empleado, ej., viáticos, capacitaciones individuales).
Descripción: description (detalle del gasto).
💡 Consideraciones para Futuro ETL y KPIs:
Costo Total de RRHH: Al sumar estos costos a los de nómina, se obtiene una visión más holística de la inversión en capital humano.
Análisis de Inversión en Personal: Identificar y cuantificar gastos en áreas como capacitación, reclutamiento, bienestar, beneficios no monetarios.
Dependencia de costs_and_expenses_categories: Crucial para desglosar y entender la naturaleza de estos gastos.

🏷️ *11. costs_and_expenses_categories (Categorías de Costos y Gastos)*

✨ Organización y Clasificación de Inversiones en RRHH ✨

Esta tabla es un maestro para clasificar los registros de costs_and_expenses, permitiendo una categorización significativa de los gastos relacionados con RRHH.

🔑 Aspectos Clave Potenciales para el Análisis de RRHH:
Identificador de Categoría: id, name (ej., "Capacitación y Desarrollo", "Reclutamiento y Selección", "Beneficios al Empleado", "Gastos de Viaje").
Tipo de Categoría: type (ej., "Operacional", "Administrativo").
💡 Consideraciones para Futuro ETL y KPIs:
Desglose de Gastos: Es fundamental para segmentar los costs_and_expenses y entender dónde se está invirtiendo el presupuesto de RRHH.
Presupuesto vs. Gasto Real: Permite comparar el gasto real en cada categoría con los presupuestos asignados.

📍 *12. headquarters (Sedes / Ubicaciones)*

✨ La Dimensión Geográfica y Estructural de la Fuerza Laboral ✨

Esta tabla contiene los detalles de las diferentes ubicaciones físicas o sedes de la empresa.

🔑 Aspectos Clave Potenciales para el Análisis de RRHH:
Identificación de Sede: id, name (nombre de la sede, ej., "Oficina Principal", "Fábrica Bogotá", "Sede Medellín").
Detalles Geográficos: address, city, region, country.
💡 Consideraciones para Futuro ETL y KPIs:
Análisis Demográfico y Geográfico: Se une directamente con employees (a través de headquarter_id) para analizar la distribución de personal por ubicación.
KPIs Segmentados por Sede: Permite realizar análisis comparativos de rendimiento, rotación, costos de nómina, etc., entre diferentes sedes.

🎁 *13. coupons (Cupones)*

✨ Herramienta Potencial para Programas de Beneficios e Incentivos ✨

Aunque no es una tabla intrínsecamente de RRHH, si la empresa utiliza cupones como parte de sus programas de beneficios, incentivos o bienestar para empleados, esta tabla es relevante.

🔑 Aspectos Clave Potenciales para el Análisis de RRHH:
Detalles del Cupón: id, code, value (valor monetario o de descuento).
Tipo y Validez: type (ej., "Descuento", "Regalo"), valid_from, valid_until (periodo de uso).
Asignación a Empleado (Importante): assigned_to_employee_id (si esta columna existe, es clave para vincular el cupón a un empleado específico).
💡 Consideraciones para Futuro ETL y KPIs:
Análisis de Beneficios No Salariales: Si los cupones son un beneficio, se puede analizar su distribución, costo total para la empresa y la participación de los empleados.
Eficacia de Programas de Incentivos: Evaluar el uso de cupones para medir el éxito de programas de incentivos.

---


#*Gráficos, Análisis y ciencia de datos*#

---

In [26]:
# Script para generar gráficos BI con Plotly Express (Distribución de Empleados por Grupos de Edad)

import pandas as pd
import plotly.express as px
from datetime import datetime
from db_connection import get_engine, get_connection 

print("\n--- Bloque 12 (Revisado con Colores Nuevos): Generación de Gráfico BI - Distribución de Empleados por Grupos de Edad ---")

conexion = None
try:
    print("Estableciendo conexión a la base de datos para la visualización...")
    engine = get_engine()
    conexion = get_connection()
    print("✅ Conexión a la base de datos establecida.")

    # 1. Cargar la columna 'birth_date' de la tabla 'employees'
    print("\nCargando columna 'birth_date' de la tabla 'employees'...")
    df_employees = pd.read_sql("SELECT birth_date FROM employees;", engine)
    print(f"DataFrame de 'employees' cargado. Registros: {len(df_employees)}")

    # 2. Limpiar y preparar los datos
    # Convertir 'birth_date' a formato datetime y manejar nulos
    df_employees['birth_date'] = pd.to_datetime(df_employees['birth_date'], errors='coerce')

    # Eliminar filas donde la fecha de nacimiento es nula, ya que no podemos calcular la edad
    df_employees.dropna(subset=['birth_date'], inplace=True)

    if df_employees.empty:
        print("❌ No hay datos válidos para generar el gráfico después de la limpieza de fechas de nacimiento.")
        raise ValueError("No hay datos suficientes para graficar.")

    # Calcular la edad de los empleados
    today = datetime.now()
    df_employees['age'] = df_employees['birth_date'].apply(lambda dob: today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day)))

    # 3. Crear rangos de edad
    # Definir los límites de los rangos de edad (ej. 20-29, 30-39, etc.)
    min_age = df_employees['age'].min() if not df_employees.empty else 0
    max_age = df_employees['age'].max() if not df_employees.empty else 100

    start_bin = (min_age // 10) * 10 if min_age >= 10 else 0
    
    bins = range(start_bin, max_age + 10, 10) # Rango de 10 en 10 años
    labels = [f'{i}-{i+9}' for i in bins[:-1]]
    
    # Crear la columna 'age_group'
    df_employees['age_group'] = pd.cut(df_employees['age'], bins=bins, labels=labels, right=False, include_lowest=True)

    # Rellenar cualquier grupo de edad nulo (si la edad cae fuera de los bins definidos)
    df_employees['age_group'] = df_employees['age_group'].cat.add_categories('Otros/No Definido').fillna('Otros/No Definido')

    print("\nConteo de empleados por grupo de edad:")
    age_counts = df_employees['age_group'].value_counts().reset_index()
    age_counts.columns = ['Age_Group', 'Employee_Count']
    # Ordenar por el orden de los rangos de edad definidos
    age_counts['Age_Group'] = pd.Categorical(age_counts['Age_Group'], categories=labels + ['Otros/No Definido'], ordered=True)
    age_counts = age_counts.sort_values('Age_Group')

    print(age_counts)

    # 4. Generar el histograma interactivo con Plotly Express
    print("\nGenerando histograma interactivo de 'Distribución de Empleados por Grupos de Edad' con nuevos colores...")
    fig = px.bar(age_counts,
                 x='Age_Group',
                 y='Employee_Count',
                 title='Distribución de Empleados por Grupos de Edad',
                 labels={
                     'Age_Group': 'Grupo de Edad',
                     'Employee_Count': 'Número de Empleados'
                 },
                 text='Employee_Count', # Muestra el número de empleados en la barra
                 color='Employee_Count', # Colorea las barras según el conteo
                 color_continuous_scale=px.colors.sequential.Plotly3 # ¡NUEVA PALETA DE COLORES!
                )

    fig.update_traces(texttemplate='%{text}', textposition='outside')
    fig.update_layout(xaxis_title="Grupo de Edad",
                      yaxis_title="Número de Empleados",
                      title_x=0.5, # Centrar el título
                      hovermode="x unified"
                     )

    # 5. Mostrar el gráfico
    fig.show()
    print("✅ Gráfico 'Distribución de Empleados por Grupos de Edad' generado y mostrado con nuevos colores.")

except ValueError as ve:
    print(f"\n❌❌ Error de datos para graficar: {ve}")
except Exception as e:
    print(f"\n❌❌ Ocurrió un error general: {e}")
    print("Asegúrate de que la tabla 'employees' existe y tiene la columna 'birth_date'.")
    print("Verifica también que 'birth_date' contenga fechas válidas.")
finally:
    if conexion and conexion.is_connected():
        conexion.close()
        print("✅ Conexión a la base de datos cerrada.")

print("\n--- Fin del Bloque de Generación de Gráficos ---")


--- Bloque 12 (Revisado con Colores Nuevos): Generación de Gráfico BI - Distribución de Empleados por Grupos de Edad ---
Estableciendo conexión a la base de datos para la visualización...
✅ Conexión exitosa a la base de datos
✅ Conexión a la base de datos establecida.

Cargando columna 'birth_date' de la tabla 'employees'...
DataFrame de 'employees' cargado. Registros: 38

Conteo de empleados por grupo de edad:
           Age_Group  Employee_Count
3              10-19               1
0              20-29              19
1              30-39              12
2              40-49               6
4  Otros/No Definido               0

Generando histograma interactivo de 'Distribución de Empleados por Grupos de Edad' con nuevos colores...


✅ Gráfico 'Distribución de Empleados por Grupos de Edad' generado y mostrado con nuevos colores.
✅ Conexión a la base de datos cerrada.

--- Fin del Bloque de Generación de Gráficos ---


In [27]:
# Script para generar gráficos BI con Plotly Express (Distribución de Empleados por Tipo de Documento de Identidad)

import pandas as pd
import plotly.express as px
from db_connection import get_engine, get_connection # Asegúrate de que este módulo esté accesible

print("\n--- Bloque 13: Generación de Gráfico BI - Distribución de Empleados por Tipo de Documento de Identidad ---")

conexion = None
try:
    print("Estableciendo conexión a la base de datos para la visualización...")
    engine = get_engine()
    conexion = get_connection()
    print("✅ Conexión a la base de datos establecida.")

    # 1. Cargar la columna 'identity_document_type' de la tabla 'employees'
    print("\nCargando columna 'identity_document_type' de la tabla 'employees'...")
    df_employees = pd.read_sql("SELECT identity_document_type FROM employees;", engine)
    print(f"DataFrame de 'employees' cargado. Registros: {len(df_employees)}")

    # 2. Limpiar y preparar los datos
    # Rellenar valores nulos o vacíos en 'identity_document_type' con 'Desconocido'
    # También limpiar espacios en blanco si los hubiera
    df_employees['identity_document_type'] = df_employees['identity_document_type'].astype(str).str.strip().replace('', 'Desconocido').fillna('Desconocido')

    # 3. Contar la frecuencia de cada tipo de documento
    print("\nContando empleados por tipo de documento de identidad...")
    document_type_counts = df_employees['identity_document_type'].value_counts().reset_index()
    document_type_counts.columns = ['Document_Type', 'Employee_Count']

    print("Conteo de empleados por tipo de documento:")
    print(document_type_counts)

    # 4. Generar el gráfico de torta interactivo con Plotly Express
    print("\nGenerando gráfico de torta interactivo de 'Distribución de Empleados por Tipo de Documento de Identidad'...")
    fig = px.pie(document_type_counts,
                 values='Employee_Count',
                 names='Document_Type',
                 title='Distribución de Empleados por Tipo de Documento de Identidad',
                 color_discrete_sequence=px.colors.qualitative.Pastel, # Una paleta de colores suave y bonita
                 hole=0.4 # Crea un gráfico de dona para un mejor impacto visual
                )

    fig.update_traces(textinfo='percent+label', # Muestra el porcentaje y la etiqueta en las rebanadas
                      hovertemplate='<b>%{label}</b><br>Empleados: %{value}<br>Porcentaje: %{percent:.1%}<extra></extra>' # Información detallada al pasar el ratón
                     )
    fig.update_layout(title_x=0.5, # Centrar el título
                      legend_title_text='Tipo de Documento') # Título para la leyenda

    # 5. Mostrar el gráfico
    fig.show()
    print("✅ Gráfico 'Distribución de Empleados por Tipo de Documento de Identidad' generado y mostrado.")

except Exception as e:
    print(f"\n❌❌ Ocurrió un error general: {e}")
    print("Asegúrate de que la tabla 'employees' existe y tiene la columna 'identity_document_type'.")
finally:
    if conexion and conexion.is_connected():
        conexion.close()
        print("✅ Conexión a la base de datos cerrada.")

print("\n--- Fin del Bloque de Generación de Gráficos ---")


--- Bloque 13: Generación de Gráfico BI - Distribución de Empleados por Tipo de Documento de Identidad ---
Estableciendo conexión a la base de datos para la visualización...
✅ Conexión exitosa a la base de datos
✅ Conexión a la base de datos establecida.

Cargando columna 'identity_document_type' de la tabla 'employees'...
DataFrame de 'employees' cargado. Registros: 38

Contando empleados por tipo de documento de identidad...
Conteo de empleados por tipo de documento:
           Document_Type  Employee_Count
0   Cédula de ciudadanía              36
1  Cédula de extranjería               2

Generando gráfico de torta interactivo de 'Distribución de Empleados por Tipo de Documento de Identidad'...


✅ Gráfico 'Distribución de Empleados por Tipo de Documento de Identidad' generado y mostrado.
✅ Conexión a la base de datos cerrada.

--- Fin del Bloque de Generación de Gráficos ---


In [28]:
# Script para generar gráficos BI con Plotly Express (Salario Promedio por Cargo)

import pandas as pd
import json # Necesario para aplanar JSON si 'name' aún lo fuera
import plotly.express as px
from db_connection import get_engine, get_connection # Asegúrate de que este módulo esté accesible

print("\n--- Bloque 14: Generación de Gráfico BI - Salario Promedio por Cargo ---")

conexion = None
cursor = None
try:
    print("Estableciendo conexión a la base de datos para la visualización y posible aplanamiento...")
    conexion = get_connection()
    cursor = conexion.cursor()
    engine = get_engine()
    print("✅ Conexión y cursor reestablecidos exitosamente.")
    print("✅ Engine de base de datos obtenido.")

    # --- PASO OPCIONAL/CRÍTICO: Si 'name' en 'employee_positions' ES JSON, modificamos su tipo ---
    # Revisamos la descripción de 'employee_positions' que me diste al principio:
    # name: varchar(255) (Nombre de la posición)
    # Según esto, 'name' ya es VARCHAR, así que no deberíamos necesitar ALTER TABLE ni aplanamiento JSON.
    # Si por alguna razón la columna 'name' hubiera sido JSON, se necesitaría este bloque:
    # print("\nVerificando y modificando columna 'name' en 'employee_positions' si es JSON...")
    # try:
    #     cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
    #     alter_table_sql = "ALTER TABLE employee_positions MODIFY COLUMN name VARCHAR(255) NULL;"
    #     cursor.execute(alter_table_sql)
    #     conexion.commit()
    #     print("✅ Columna 'name' modificada a VARCHAR(255) NULL exitosamente (si era JSON).")
    # except Exception as e:
    #     conexion.rollback()
    #     print(f"⚠️ Advertencia: No se pudo modificar la columna 'name' (ya es VARCHAR o error): {e}")
    # finally:
    #     cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")

    # 1. Cargar las columnas 'name' y 'salary' de la tabla 'employee_positions'
    print("\nCargando columnas 'name' y 'salary' de la tabla 'employee_positions'...")
    df_positions = pd.read_sql("SELECT name, salary FROM employee_positions;", engine)
    print(f"DataFrame de 'employee_positions' cargado. Registros: {len(df_positions)}")

    # 2. Limpiar y preparar los datos
    # Rellenar nombres de cargo nulos con 'Cargo Desconocido'
    df_positions['name'] = df_positions['name'].fillna('Cargo Desconocido').astype(str).str.strip()

    # Eliminar filas con salarios nulos o no válidos para el cálculo del promedio
    df_positions.dropna(subset=['salary'], inplace=True)

    if df_positions.empty:
        print("❌ No hay datos válidos (salarios o nombres de cargo) para generar el gráfico.")
        raise ValueError("No hay datos suficientes para graficar.")

    # 3. Calcular el salario promedio por cada cargo
    print("\nCalculando el salario promedio por cargo...")
    avg_salary_by_position = df_positions.groupby('name')['salary'].mean().reset_index()
    avg_salary_by_position.columns = ['Position_Name', 'Average_Salary']

    # Redondear el salario promedio para mejor visualización
    avg_salary_by_position['Average_Salary'] = avg_salary_by_position['Average_Salary'].round(2)

    # Opcional: Ordenar los cargos por salario promedio para una mejor lectura
    avg_salary_by_position = avg_salary_by_position.sort_values('Average_Salary', ascending=False)

    print("Salario promedio por cargo:")
    print(avg_salary_by_position.head())

    # 4. Generar el gráfico de barras interactivo con Plotly Express
    print("\nGenerando gráfico de barras interactivo de 'Salario Promedio por Cargo'...")
    fig = px.bar(avg_salary_by_position,
                 x='Position_Name',
                 y='Average_Salary',
                 title='Salario Promedio por Cargo',
                 labels={
                     'Position_Name': 'Cargo / Posición',
                     'Average_Salary': 'Salario Promedio'
                 },
                 text_auto='.2s', # Muestra el valor del salario promedio en las barras
                 color='Average_Salary', # Colorea las barras según el salario promedio
                 color_continuous_scale=px.colors.sequential.Plasma # Puedes probar otras como 'Viridis', 'Inferno', 'Cividis'
                )

    fig.update_traces(textposition='outside') # Coloca el texto fuera de las barras para evitar que se solape
    fig.update_layout(xaxis_title="Cargo / Posición",
                      yaxis_title="Salario Promedio",
                      title_x=0.5, # Centrar el título
                      hovermode="x unified"
                     )

    # 5. Mostrar el gráfico
    fig.show()
    print("✅ Gráfico 'Salario Promedio por Cargo' generado y mostrado.")

except ValueError as ve:
    print(f"\n❌❌ Error de datos para graficar: {ve}")
except Exception as e:
    print(f"\n❌❌ Ocurrió un error general: {e}")
    print("Asegúrate de que la tabla 'employee_positions' existe y tiene las columnas 'name' y 'salary'.")
    print("Verifica también que 'salary' contenga valores numéricos válidos.")
finally:
    if cursor is not None:
        try:
            # Asegurarse de que las FKs se re-habilitan si se deshabilitaron (aunque aquí no se espera)
            cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
        except Exception as e:
            pass # Ignorar si no se pudo re-habilitar (ya estaban habilitadas)
    if conexion and conexion.is_connected():
        conexion.close()
        print("✅ Conexión a la base de datos cerrada.")

print("\n--- Fin del Bloque de Generación de Gráficos ---")


--- Bloque 14: Generación de Gráfico BI - Salario Promedio por Cargo ---
Estableciendo conexión a la base de datos para la visualización y posible aplanamiento...
✅ Conexión exitosa a la base de datos
✅ Conexión y cursor reestablecidos exitosamente.
✅ Engine de base de datos obtenido.

Cargando columnas 'name' y 'salary' de la tabla 'employee_positions'...
DataFrame de 'employee_positions' cargado. Registros: 15

Calculando el salario promedio por cargo...
Salario promedio por cargo:
          Position_Name  Average_Salary
8               Gerente       7500000.0
5         Desarrollador       4725000.0
6         Diseñador Web       2800000.0
12   Tesorero principal       2500000.0
3   Auxiliar de soporte       2500000.0

Generando gráfico de barras interactivo de 'Salario Promedio por Cargo'...


✅ Gráfico 'Salario Promedio por Cargo' generado y mostrado.
✅ Conexión a la base de datos cerrada.

--- Fin del Bloque de Generación de Gráficos ---


In [29]:
# Script para generar gráficos BI con Plotly Express (Conteo de Empleados por Posición)

import pandas as pd
import plotly.express as px
from db_connection import get_engine, get_connection # Asegúrate de que este módulo esté accesible

print("\n--- Bloque 15: Generación de Gráfico BI - Conteo de Empleados por Posición ---")

conexion = None
try:
    print("Estableciendo conexión a la base de datos para la visualización...")
    engine = get_engine()
    conexion = get_connection()
    print("✅ Conexión a la base de datos establecida.")

    # 1. Cargar datos de ambas tablas y unirlas
    print("\nCargando datos de 'employees' y 'employee_positions' y uniendo por el ID de posición...")
    query = """
    SELECT
        e.id AS employee_id,
        ep.name AS position_name
    FROM
        employees AS e
    JOIN
        employee_positions AS ep ON e.employee_position_id = ep.id;
    """
    df_employee_positions = pd.read_sql(query, engine)
    print(f"DataFrame combinado cargado. Registros: {len(df_employee_positions)}")

    # 2. Limpiar y preparar los datos
    # Rellenar nombres de posición nulos con 'Posición Desconocida'
    df_employee_positions['position_name'] = df_employee_positions['position_name'].fillna('Posición Desconocida').astype(str).str.strip()

    # 3. Contar empleados por cada nombre de posición
    print("\nContando el número de empleados por cada posición...")
    employees_count_by_position = df_employee_positions['position_name'].value_counts().reset_index()
    employees_count_by_position.columns = ['Position_Name', 'Employee_Count']

    # Opcional: Ordenar por el número de empleados (descendente)
    employees_count_by_position = employees_count_by_position.sort_values('Employee_Count', ascending=False)

    print("Conteo de empleados por posición:")
    print(employees_count_by_position.head())

    # 4. Generar el gráfico de barras interactivo con Plotly Express
    print("\nGenerando gráfico de barras interactivo de 'Conteo de Empleados por Posición'...")
    fig = px.bar(employees_count_by_position,
                 x='Position_Name',
                 y='Employee_Count',
                 title='Número de Empleados por Posición',
                 labels={
                     'Position_Name': 'Posición / Cargo',
                     'Employee_Count': 'Número de Empleados'
                 },
                 text='Employee_Count', # Muestra el número de empleados encima de las barras
                 color='Employee_Count', # Colorea las barras según el conteo
                 color_continuous_scale=px.colors.sequential.Bluyl # Otra paleta de colores bonita
                )

    fig.update_traces(texttemplate='%{text}', textposition='outside')
    fig.update_layout(xaxis_title="Posición / Cargo",
                      yaxis_title="Número de Empleados",
                      title_x=0.5, # Centrar el título
                      hovermode="x unified"
                     )

    # 5. Mostrar el gráfico
    fig.show()
    print("✅ Gráfico 'Número de Empleados por Posición' generado y mostrado.")

except Exception as e:
    print(f"\n❌❌ Ocurrió un error general: {e}")
    print("Asegúrate de que las tablas 'employees' y 'employee_positions' existen, y que la columna 'employee_position_id' en 'employees' y 'id'/'name' en 'employee_positions' son correctas.")
finally:
    if conexion and conexion.is_connected():
        conexion.close()
        print("✅ Conexión a la base de datos cerrada.")

print("\n--- Fin del Bloque de Generación de Gráficos ---")


--- Bloque 15: Generación de Gráfico BI - Conteo de Empleados por Posición ---
Estableciendo conexión a la base de datos para la visualización...
✅ Conexión exitosa a la base de datos
✅ Conexión a la base de datos establecida.

Cargando datos de 'employees' y 'employee_positions' y uniendo por el ID de posición...
DataFrame combinado cargado. Registros: 38

Contando el número de empleados por cada posición...
Conteo de empleados por posición:
              Position_Name  Employee_Count
0             Desarrollador               8
1                   General               7
2  Asistente Administrativo               7
3             Diseñador Web               4
4                   Gerente               3

Generando gráfico de barras interactivo de 'Conteo de Empleados por Posición'...


✅ Gráfico 'Número de Empleados por Posición' generado y mostrado.
✅ Conexión a la base de datos cerrada.

--- Fin del Bloque de Generación de Gráficos ---


In [30]:
# Script para generar gráficos BI con Plotly Express (Conteo de Empleados por Estado con Número Absoluto)

import pandas as pd
import plotly.express as px
from db_connection import get_engine, get_connection # Asegúrate de que este módulo esté accesible

print("\n--- Bloque 16 (Conteo Absoluto): Generación de Gráfico BI - Conteo de Empleados por Estado ---")

conexion = None
try:
    print("Estableciendo conexión a la base de datos para la visualización...")
    engine = get_engine()
    conexion = get_connection()
    print("✅ Conexión a la base de datos establecida.")

    # 1. Cargar la columna de estado de la tabla 'employees'
    # <<< ¡ATENCIÓN AQUÍ! >>>
    # SI LA COLUMNA CON LOS VALORES 0 y 1 NO SE LLAMA 'status', CÁMBIALA EN LA SIGUIENTE LÍNEA.
    # Por ejemplo, si se llama 'is_active', sería "SELECT is_active FROM employees;"
    print("\nCargando columna 'status' (o similar con 0/1) de la tabla 'employees'...")
    df_employees_status = pd.read_sql("SELECT status FROM employees;", engine)
    print(f"DataFrame de 'employees' cargado. Registros: {len(df_employees_status)}")

    # 2. Mapear los valores 0 y 1 a etiquetas significativas
    print("\nMapeando valores de estado (0=Ex-empleado, 1=Empleado)...")
    status_mapping = {
        0: 'Ex-empleado',
        1: 'Empleado'
    }
    # Aplicar el mapeo. Cualquier otro valor se convertirá en 'Desconocido'
    df_employees_status['Employee_Status'] = df_employees_status['status'].map(status_mapping).fillna('Desconocido')

    # 3. Contar la frecuencia de cada estado mapeado
    print("\nContando empleados por estado de empleo (mapeado)...")
    status_counts = df_employees_status['Employee_Status'].value_counts().reset_index()
    status_counts.columns = ['Employee_Status', 'Employee_Count']

    print("Conteo de empleados por estado de empleo (mapeado):")
    print(status_counts)

    # 4. Generar el gráfico de torta interactivo con Plotly Express
    print("\nGenerando gráfico de torta interactivo de 'Conteo de Empleados por Estado' (¡con número absoluto!)...")
    fig = px.pie(status_counts,
                 values='Employee_Count',
                 names='Employee_Status',
                 title='Distribución de Empleados por Estado de Empleo',
                 color_discrete_sequence=px.colors.qualitative.Pastel, # Una paleta más suave y bonita
                 hole=0.4 # Gráfico de dona
                )

    # <<<< ¡CAMBIO CLAVE AQUÍ! >>>>
    # Añadimos 'value' a 'textinfo' para mostrar el número absoluto
    fig.update_traces(textinfo='percent+label+value',
                      hovertemplate='<b>%{label}</b><br>Empleados: %{value}<br>Porcentaje: %{percent:.1%}<extra></extra>'
                     )
    fig.update_layout(title_x=0.5,
                      legend_title_text='Estado del Empleado')

    # 5. Mostrar el gráfico
    fig.show()
    print("✅ Gráfico 'Conteo de Empleados por Estado' generado y mostrado con número absoluto y mapeo de valores.")

except Exception as e:
    print(f"\n❌❌ Ocurrió un error general: {e}")
    print("¡ATENCIÓN! El error puede indicar que la columna de estado NO EXISTE o no contiene valores numéricos 0 y 1.")
    print("Por favor, verifica el nombre exacto de la columna que indica el estado del empleado (0/1) en tu tabla 'employees'.")
    print("Si el problema persiste, es posible que la columna no contenga solo 0s y 1s o tenga un tipo de dato inesperado.")
finally:
    if conexion and conexion.is_connected():
        conexion.close()
        print("✅ Conexión a la base de datos cerrada.")

print("\n--- Fin del Bloque de Generación de Gráficos ---")


--- Bloque 16 (Conteo Absoluto): Generación de Gráfico BI - Conteo de Empleados por Estado ---
Estableciendo conexión a la base de datos para la visualización...
✅ Conexión exitosa a la base de datos
✅ Conexión a la base de datos establecida.

Cargando columna 'status' (o similar con 0/1) de la tabla 'employees'...
DataFrame de 'employees' cargado. Registros: 38

Mapeando valores de estado (0=Ex-empleado, 1=Empleado)...

Contando empleados por estado de empleo (mapeado)...
Conteo de empleados por estado de empleo (mapeado):
  Employee_Status  Employee_Count
0     Ex-empleado              21
1        Empleado              17

Generando gráfico de torta interactivo de 'Conteo de Empleados por Estado' (¡con número absoluto!)...


✅ Gráfico 'Conteo de Empleados por Estado' generado y mostrado con número absoluto y mapeo de valores.
✅ Conexión a la base de datos cerrada.

--- Fin del Bloque de Generación de Gráficos ---


📊 Tasa de Rotación de Empleados (Anual)

Este KPI mide la proporción de empleados que abandonan la empresa en un período determinado (en este caso, anualmente), en relación con el número total de empleados. Es un indicador clave de la salud del clima laboral y la capacidad de la empresa para retener talento.

In [31]:
# Script para generar gráficos BI con Plotly Express (Tasa de Rotación Anual de Empleados - CON DATOS SIMULADOS MEJORADOS)

import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta # Importamos timedelta para simular fechas pasadas
from db_connection import get_engine, get_connection

print("\n--- Bloque 17 (MEJORADO - Datos Simulados): Generación de Gráfico BI - Tasa de Rotación Anual ---")

conexion = None
try:
    print("Estableciendo conexión a la base de datos para el cálculo de rotación...")
    engine = get_engine()
    conexion = get_connection()
    print("✅ Conexión a la base de datos establecida.")

    # 1. Cargar las columnas necesarias de la tabla 'employees'
    print("\nCargando 'id', 'created_at', y 'status' de la tabla 'employees'...")
    df_employees = pd.read_sql("SELECT id, created_at, status FROM employees;", engine)
    print(f"DataFrame de 'employees' cargado. Registros: {len(df_employees)}")

    # 2. Limpiar y preparar los datos
    df_employees['created_at'] = pd.to_datetime(df_employees['created_at'], errors='coerce')
    df_employees['status'] = pd.to_numeric(df_employees['status'], errors='coerce').fillna(1).astype(int)

    df_employees.dropna(subset=['created_at'], inplace=True)

    if df_employees.empty:
        print("❌ No hay datos válidos (fechas de creación o estado) para generar el gráfico.")
        raise ValueError("No hay datos suficientes para graficar la rotación.")

    # --- Lógica de cálculo de rotación más precisa ---
    print("\nCalculando la tasa de rotación anual de empleados (metodología mejorada con simulación histórica)...")

    # === MEJORA EN LA SIMULACIÓN DE TERMINATION_DATE ===
    # Si un empleado tiene status 0 (ex-empleado), asignaremos una termination_date
    # que sea un año después de su created_at, o la fecha actual si eso lo lleva al futuro.
    # Esto es para distribuir las "salidas" en años pasados y tener datos para el gráfico.
    
    current_year = datetime.now().year
    
    def simulate_termination_date(row):
        if row['status'] == 0: # Si es ex-empleado
            # Simular que se fue un año después de su contratación
            simulated_term_date = row['created_at'] + timedelta(days=365) 
            # Asegurarse de que no sea en el futuro si la contratación fue muy reciente
            if simulated_term_date.year > current_year:
                return datetime(current_year, 12, 31) # Si es muy reciente, asigna fin del año actual
            return simulated_term_date
        return pd.NaT # Si es activo, no tiene fecha de terminación
    
    df_employees['termination_date'] = df_employees.apply(simulate_termination_date, axis=1)

    # 3. Calcular la rotación anual basada en un método más estándar
    # A. Contar Salidas por Año
    # Ahora usamos la 'termination_date' simulada para el año de salida
    departures_annual = df_employees[df_employees['status'] == 0].copy()
    departures_annual['exit_year'] = departures_annual['termination_date'].dt.year
    departures_count = departures_annual.groupby('exit_year').size().reset_index(name='Departures')
    departures_count.columns = ['Year', 'Departures']

    # B. Calcular Plantilla Promedio (Headcount)
    # Para la base, vamos a calcular una aproximación del headcount al inicio de cada año.
    # Asumimos que un empleado contribuye al headcount desde su `created_at` hasta su `termination_date` (o hasta ahora si está activo).
    
    # Crear una lista de todos los años relevantes
    min_year = df_employees['created_at'].dt.year.min()
    max_year = datetime.now().year # Usamos el año actual como el último año para el cálculo de headcount
    all_years = range(min_year, max_year + 1)
    
    headcount_data = []
    for year in all_years:
        # Empleados que están activos (status=1) en algún punto de este año
        # O que su termination_date no fue antes del inicio de este año
        
        # Empleados activos al inicio del año (contratados antes del 1 de enero del año Y, y no terminados antes de esa fecha)
        employees_at_start_of_year = df_employees[
            (df_employees['created_at'].dt.year < year) | # Contratados en años anteriores
            ((df_employees['created_at'].dt.year == year) & (df_employees['created_at'].dt.month == 1) & (df_employees['created_at'].dt.day == 1)) # O el 1 de enero del año
        ].copy()
        
        # Filtrar aquellos que no han terminado antes del inicio del año
        employees_at_start_of_year = employees_at_start_of_year[
            (employees_at_start_of_year['status'] == 1) | # Siguen activos
            (employees_at_start_of_year['termination_date'].dt.year >= year) | # O terminaron en este año o después
            (pd.isna(employees_at_start_of_year['termination_date'])) # O no tienen fecha de terminación (activos)
        ]

        headcount_data.append({'Year': year, 'Headcount': len(employees_at_start_of_year.id.unique())})
    
    df_headcount = pd.DataFrame(headcount_data)
    
    # Unir datos de salidas con el headcount
    rotation_summary = pd.merge(df_headcount, departures_count, on='Year', how='left').fillna(0)
    
    # Calcular Tasa de Rotación
    rotation_summary['Rotation_Rate'] = rotation_summary.apply(
        lambda row: (row['Departures'] / row['Headcount']) * 100 if row['Headcount'] > 0 else 0, axis=1
    )
    rotation_summary['Rotation_Rate'] = rotation_summary['Rotation_Rate'].round(2)

    rotation_summary = rotation_summary[rotation_summary['Headcount'] > 0] # Filtrar años sin plantilla
    
    print("Datos de rotación anual (simulación mejorada):")
    print(rotation_summary)

    if rotation_summary.empty:
        print("❌ No hay suficientes datos para calcular la rotación anual con las columnas proporcionadas.")
        raise ValueError("No hay datos de rotación para graficar.")

    # 4. Generar el gráfico de líneas interactivo
    print("\nGenerando gráfico de líneas de 'Tasa de Rotación Anual de Empleados'...")
    fig = px.line(rotation_summary,
                  x='Year',
                  y='Rotation_Rate',
                  title='Tasa de Rotación Anual de Empleados (Simulada)',
                  labels={
                      'Year': 'Año',
                      'Rotation_Rate': 'Tasa de Rotación (%)'
                  },
                  markers=True,
                  text='Rotation_Rate',
                  color_discrete_sequence=['#FF6347']
                 )

    fig.update_traces(texttemplate='%{text:.2f}%', textposition='top center')
    fig.update_layout(xaxis_title="Año",
                      yaxis_title="Tasa de Rotación (%)",
                      title_x=0.5,
                      hovermode="x unified",
                      yaxis_range=[0, rotation_summary['Rotation_Rate'].max() * 1.2] if not rotation_summary.empty else [0, 100]
                     )

    fig.show()
    print("✅ Gráfico 'Tasa de Rotación Anual de Empleados' generado y mostrado.")

except ValueError as ve:
    print(f"\n❌❌ Error de datos para graficar: {ve:}")
except Exception as e:
    print(f"\n❌❌ Ocurrió un error general: {e}")
    print("Asegúrate de que la tabla 'employees' existe y tiene las columnas 'id', 'created_at', y 'status'.")
    print("Verifica que 'created_at' son fechas válidas y 'status' son 0 o 1.")
    print("Si tu columna de estado no se llama 'status', cámbiala en el script.")
finally:
    if conexion and conexion.is_connected():
        conexion.close()
        print("✅ Conexión a la base de datos cerrada.")

print("\n--- Fin del Bloque de Generación de Gráficos ---")


--- Bloque 17 (MEJORADO - Datos Simulados): Generación de Gráfico BI - Tasa de Rotación Anual ---
Estableciendo conexión a la base de datos para el cálculo de rotación...
✅ Conexión exitosa a la base de datos
✅ Conexión a la base de datos establecida.

Cargando 'id', 'created_at', y 'status' de la tabla 'employees'...
DataFrame de 'employees' cargado. Registros: 38

Calculando la tasa de rotación anual de empleados (metodología mejorada con simulación histórica)...
Datos de rotación anual (simulación mejorada):
   Year  Headcount  Departures  Rotation_Rate
1  2022         12         8.0          66.67
2  2023         22        11.0          50.00
3  2024         14         2.0          14.29
4  2025         17         0.0           0.00

Generando gráfico de líneas de 'Tasa de Rotación Anual de Empleados'...


✅ Gráfico 'Tasa de Rotación Anual de Empleados' generado y mostrado.
✅ Conexión a la base de datos cerrada.

--- Fin del Bloque de Generación de Gráficos ---


📊 Antigüedad Promedio de Empleados

Este gráfico te ayudará a entender cuánto tiempo suelen permanecer tus empleados en la empresa. Una alta antigüedad promedio puede indicar alta retención y satisfacción.

Pregunta de Negocio: ¿Cuál es la antigüedad promedio de nuestros empleados y cómo se distribuye esta antigüedad?

In [32]:
# Script para generar gráficos BI con Plotly Express (Antigüedad Promedio por Cargo)

import pandas as pd
import plotly.express as px
from datetime import datetime
from db_connection import get_engine, get_connection # Asegúrate de que este módulo esté accesible

print("\n--- Bloque 18 (Opción B): Generación de Gráfico BI - Antigüedad Promedio por Cargo ---")

conexion = None
try:
    print("Estableciendo conexión a la base de datos para el cálculo de antigüedad por cargo...")
    engine = get_engine()
    conexion = get_connection()
    print("✅ Conexión a la base de datos establecida.")

    # 1. Cargar datos de ambas tablas y unirlas
    print("\nCargando 'created_at' de 'employees' y 'name' de 'employee_positions'...")
    query = """
    SELECT
        e.created_at,
        ep.name AS position_name
    FROM
        employees AS e
    JOIN
        employee_positions AS ep ON e.employee_position_id = ep.id;
    """
    df_employees_tenure = pd.read_sql(query, engine)
    print(f"DataFrame combinado cargado. Registros: {len(df_employees_tenure)}")

    # 2. Limpiar y preparar los datos
    df_employees_tenure['created_at'] = pd.to_datetime(df_employees_tenure['created_at'], errors='coerce')

    # Rellenar nombres de cargo nulos con 'Posición Desconocida'
    df_employees_tenure['position_name'] = df_employees_tenure['position_name'].fillna('Posición Desconocida').astype(str).str.strip()

    # Eliminar filas con fechas de creación nulas
    df_employees_tenure.dropna(subset=['created_at'], inplace=True)

    if df_employees_tenure.empty:
        print("❌ No hay datos válidos (fechas de inicio de empleo o nombres de cargo) para generar el gráfico.")
        raise ValueError("No hay datos suficientes para graficar la antigüedad por cargo.")

    # 3. Calcular la antigüedad en años para cada empleado
    today = datetime.now()
    df_employees_tenure['tenure_years'] = ((today - df_employees_tenure['created_at']).dt.days / 365.25).round(1)

    # 4. Calcular la antigüedad promedio por cargo
    print("\nCalculando la antigüedad promedio por cargo...")
    avg_tenure_by_position = df_employees_tenure.groupby('position_name')['tenure_years'].mean().reset_index()
    avg_tenure_by_position.columns = ['Position_Name', 'Average_Tenure_Years']

    # Redondear el promedio para mejor visualización
    avg_tenure_by_position['Average_Tenure_Years'] = avg_tenure_by_position['Average_Tenure_Years'].round(2)

    # Opcional: Ordenar los cargos por antigüedad promedio para una mejor lectura
    avg_tenure_by_position = avg_tenure_by_position.sort_values('Average_Tenure_Years', ascending=False)

    print("Antigüedad promedio por cargo:")
    print(avg_tenure_by_position.head())

    # 5. Generar el gráfico de barras interactivo con Plotly Express
    print("\nGenerando gráfico de barras de 'Antigüedad Promedio por Cargo'...")
    fig = px.bar(avg_tenure_by_position,
                 x='Position_Name',
                 y='Average_Tenure_Years',
                 title='Antigüedad Promedio de Empleados por Cargo',
                 labels={
                     'Position_Name': 'Cargo / Posición',
                     'Average_Tenure_Years': 'Antigüedad Promedio (Años)'
                 },
                 text_auto='.2s', # Muestra el valor de antigüedad promedio en las barras
                 color='Average_Tenure_Years', # Colorea las barras según la antigüedad promedio
                 color_continuous_scale=px.colors.sequential.Agsunset # Una paleta de colores atractiva
                )

    fig.update_traces(textposition='outside')
    fig.update_layout(xaxis_title="Cargo / Posición",
                      yaxis_title="Antigüedad Promedio (Años)",
                      title_x=0.5, # Centrar el título
                      hovermode="x unified"
                     )

    # 6. Mostrar el gráfico
    fig.show()
    print("✅ Gráfico 'Antigüedad Promedio por Cargo' generado y mostrado.")

except ValueError as ve:
    print(f"\n❌❌ Error de datos para graficar: {ve}")
except Exception as e:
    print(f"\n❌❌ Ocurrió un error general: {e}")
    print("Asegúrate de que las tablas 'employees' y 'employee_positions' existen.")
    print("Verifica que 'employees' tiene 'created_at' y 'employee_position_id'.")
    print("Verifica que 'employee_positions' tiene 'id' y 'name'.")
    print("Asegúrate que 'created_at' contiene fechas válidas.")
finally:
    if conexion and conexion.is_connected():
        conexion.close()
        print("✅ Conexión a la base de datos cerrada.")

print("\n--- Fin del Bloque de Generación de Gráficos ---")


--- Bloque 18 (Opción B): Generación de Gráfico BI - Antigüedad Promedio por Cargo ---
Estableciendo conexión a la base de datos para el cálculo de antigüedad por cargo...
✅ Conexión exitosa a la base de datos
✅ Conexión a la base de datos establecida.

Cargando 'created_at' de 'employees' y 'name' de 'employee_positions'...
DataFrame combinado cargado. Registros: 38

Calculando la antigüedad promedio por cargo...
Antigüedad promedio por cargo:
                         Position_Name  Average_Tenure_Years
9                   Logístico Integral                   3.5
10                  Tesorero principal                   3.5
0   Asesor de Ventas para las fabricas                   3.4
11                     Ventas internas                   3.4
2   Auxiliar Contable y Administrativa                   3.3

Generando gráfico de barras de 'Antigüedad Promedio por Cargo'...


✅ Gráfico 'Antigüedad Promedio por Cargo' generado y mostrado.
✅ Conexión a la base de datos cerrada.

--- Fin del Bloque de Generación de Gráficos ---
